***GENERATED CODE FOR dptestsink PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '47', 'mean': '', 'stddev': '', 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '47', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfEntry'}, {'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfService'}, {'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '47', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '47', 'mean': '92546.21', 'stddev': '1758.91', 'min': '92012', 'max': '99204', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '47', 'mean': '', 'stddev': '', 'min': 'E11.3312', 'max': 'H53.9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '47', 'mean': '', 'stddev': '', 'min': 'B39.4', 'max': 'Z96.1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '47', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '47', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '47', 'mean': '', 'stddev': '', 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature_label': 'PayerId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerId')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, path):
        df.to_csv("/dbfs" + eval(config)['url'], header=True, index=False)
        return True


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run dptestsinkHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\r\n\t\t\t\tSub2.PayerId\r\n\r\nFROM \r\n\r\n(\r\n\r\n\t\tSELECT \r\n\r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\r\n\t\tpp.ProviderID,\r\n\r\n\t\tcptc.CPTCode,\r\n\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\r\n\t\td.DepartmentCategory,\r\n\r\n\t\ts.SpecialityCategory,\r\n\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\r\n\t\tfrom Transactions t\r\n\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\r\n\t\tand t.DateOfEntry between '1/1/2020' and '1/05/2020'  \r\n\r\n) as sub2\r\n\r\nINNER JOIN\r\n\r\n(\r\n\r\n\t\tSELECT\r\n\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\r\n\t\t\t t.DateOfEntry,\r\n\r\n\t\t\t count(*) as total_transactions\r\n\r\n\t\tfrom Transactions t\r\n\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\r\n\t\tWHERE t.Amount > 0.1 \r\n\r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n\r\n) as sub1\r\n\r\n on sub1.cid = sub2.ChargeDetailID\r\n\r\n where sub2.Gender is not NULL\r\n\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\r\n\t\t\t\tSub2.PayerId", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dptestsinkHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "", "stddev": "", "min": "0085689C-C880-444A-AD7E-ACAF2BD1C2B7", "max": "FD457EC1-8819-47D0-B28D-3BCF4DB749C2", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "47", "mean": "71.83", "stddev": "15.61", "min": "34", "max": "98", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "DateOfEntry", "transformation_label": "Extract Date"}], "feature": "DateOfEntry", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfEntry"}, {"transformationsData": [{"feature_label": "DateOfService", "transformation_label": "Extract Date"}], "feature": "DateOfService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfService"}, {"transformationsData": [{"feature_label": "ProviderID", "transformation_label": "String Indexer"}], "feature": "ProviderID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "", "stddev": "", "min": "03151B5D-685B-4211-B01F-7BBB1C4A8A49", "max": "CBBD4925-1757-45BC-A853-1C2CA203B9E3", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderID"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "92546.21", "stddev": "1758.91", "min": "92012", "max": "99204", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "", "stddev": "", "min": "E11.3312", "max": "H53.9", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "", "stddev": "", "min": "B39.4", "max": "Z96.1", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "", "stddev": "", "min": "Cardiology", "max": "Pulmonary ", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "", "stddev": "", "min": "Ophtamology", "max": "Ophtamology", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_to_first_post", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "47", "mean": "30.79", "stddev": "26.24", "min": "0", "max": "138", "missing": "0"}, "updatedLabel": "days_to_first_post"}, {"transformationsData": [{"feature_label": "PayerId", "transformation_label": "String Indexer"}], "feature": "PayerId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "47", "mean": "", "stddev": "", "min": "03E92275-9F7C-4D34-8701-CE4C72CDE9F7", "max": "F71F64B6-E8D9-4A9C-A7E3-A518F957DE52", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerId"}, {"feature": "ChargeDetailID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "47", "mean": "23.0", "stddev": "13.71", "min": "0.0", "max": "46.0", "missing": "0"}, "updatedLabel": "ChargeDetailID_stringinde..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "47", "mean": "0.45", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DateOfEntry_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "47", "mean": "4.0", "stddev": "0.96", "min": "2", "max": "5", "missing": "0"}, "updatedLabel": "DateOfEntry_dayofmonth"}, {"feature": "DateOfEntry_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "47", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "DateOfEntry_month"}, {"feature": "DateOfEntry_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "47", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "DateOfEntry_year"}, {"feature": "DateOfService_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "47", "mean": "5.66", "stddev": "7.26", "min": "2", "max": "29", "missing": "0"}, "updatedLabel": "DateOfService_dayofmonth"}, {"feature": "DateOfService_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "47", "mean": "2.17", "stddev": "3.43", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "DateOfService_month"}, {"feature": "DateOfService_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "47", "mean": "2019.89", "stddev": "0.31", "min": "2019", "max": "2020", "missing": "0"}, "updatedLabel": "DateOfService_year"}, {"feature": "ProviderID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "47", "mean": "3.53", "stddev": "2.89", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "ProviderID_stringindexer_..."}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "47", "mean": "1.26", "stddev": "1.5", "min": "0.0", "max": "6.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "47", "mean": "13.06", "stddev": "11.16", "min": "0.0", "max": "34.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "47", "mean": "12.13", "stddev": "10.16", "min": "0.0", "max": "32.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "47", "mean": "0.81", "stddev": "0.88", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "47", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PayerId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "47", "mean": "4.79", "stddev": "5.79", "min": "0.0", "max": "19.0", "missing": "0"}, "updatedLabel": "PayerId_stringindexer_tra..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run dptestsinkHooks.ipynb
try:
	#sinkPreExecutionHook()

	dbfssink = DBFSConnector.put(df, "{'url': '/tmp/undefined.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'source_attributes': {'AutoFE': {'FE': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '71.83', 'stddev': '15.61', 'min': '34', 'max': '98', 'missing': '0'}, 'updatedLabel': 'Age'}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateOfEntry'}, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateOfService'}, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '92546.21', 'stddev': '1758.91', 'min': '92012', 'max': '99204', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'E11.3312', 'max': 'H53.9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'B39.4', 'max': 'Z96.1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'days_to_first_post', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '30.79', 'stddev': '26.24', 'min': '0', 'max': '138', 'missing': '0'}, 'updatedLabel': 'days_to_first_post'}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '23.0', 'stddev': '13.71', 'min': '0.0', 'max': '46.0', 'missing': '0'}, 'updatedLabel': 'ChargeDetailID_stringinde...'}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}, 'updatedLabel': 'Gender_stringindexer_tran...'}, {'feature': 'DateOfEntry_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '4.0', 'stddev': '0.96', 'min': '2', 'max': '5', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_dayofmonth'}, {'feature': 'DateOfEntry_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '1.0', 'stddev': '0.0', 'min': '1', 'max': '1', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_month'}, {'feature': 'DateOfEntry_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_year'}, {'feature': 'DateOfService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '5.66', 'stddev': '7.26', 'min': '2', 'max': '29', 'missing': '0'}, 'updatedLabel': 'DateOfService_dayofmonth'}, {'feature': 'DateOfService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2.17', 'stddev': '3.43', 'min': '1', 'max': '12', 'missing': '0'}, 'updatedLabel': 'DateOfService_month'}, {'feature': 'DateOfService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2019.89', 'stddev': '0.31', 'min': '2019', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateOfService_year'}, {'feature': 'ProviderID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '3.53', 'stddev': '2.89', 'min': '0.0', 'max': '10.0', 'missing': '0'}, 'updatedLabel': 'ProviderID_stringindexer_...'}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '1.26', 'stddev': '1.5', 'min': '0.0', 'max': '6.0', 'missing': '0'}, 'updatedLabel': 'CPTCode_stringindexer_tra...'}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '13.06', 'stddev': '11.16', 'min': '0.0', 'max': '34.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis1_stringindexer...'}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '12.13', 'stddev': '10.16', 'min': '0.0', 'max': '32.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis2_stringindexer...'}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.81', 'stddev': '0.88', 'min': '0', 'max': '3', 'missing': '0'}, 'updatedLabel': 'DepartmentCategory_string...'}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}, 'updatedLabel': 'SpecialityCategory_string...'}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '4.79', 'stddev': '5.79', 'min': '0.0', 'max': '19.0', 'missing': '0'}, 'updatedLabel': 'PayerId_stringindexer_tra...'}], 'dataPercentage': '10', 'samplecount': 47, 'originalcount': 1005, 'FEGraph': [{'values': {'label': ['8406E488-7E1E-482B-80B1-4AE30E2F0CD6', '6058CDA2-9A21-4685-BA2F-8DC6AB7F8157', '5B38B5CA-A8C8-46C1-AFE3-01B32A084701', '7152B9FD-20E2-4BAA-9880-ECEEF7B7813A', '5E8A42D6-AF8A-48A6-A57C-769AA7E36D6A', '372F7455-EF3A-428E-923C-3B0465D2F474', '2071960F-CC64-4260-948F-3A0FC98E18CE', 'BE2AA60D-3002-47AB-A5AD-1FF10FA52D7A', '50503BED-86A3-4C40-9F05-A8D32F5DABE6', '7ADFBCB9-7E57-44B9-92A5-D9EA6D148794', '132E05B1-87E1-49D5-891D-7D24E6959B94', '008CD5CF-D7BE-44C6-AB9A-1EA061BE4800', 'A784EB62-A0F5-463D-BD9F-412AF036DDD7', 'C481E307-0A94-467B-BF82-53D34728427E', '43846138-A55C-4877-863F-C1AC659A3F44', '80CB21C5-3FFF-4F35-B228-4A1ABB4D2015', '6BDD5CED-1DE1-4774-BC24-03127487C3D3', '4D15A79D-1B97-461B-8994-C9DB9BBFE5FF', 'DB8DA258-E24D-43A0-BE20-9DABC58150D6', '327659EE-D609-4ED5-83FB-A87DF7EAC224', '1E6165E2-7C9B-4680-972C-72BFD783CCA6', 'A21E71B8-5267-4260-B174-EBEACF6765A6', '04027CE6-725D-45D3-B3F5-FE1684208F6F', 'BB460B17-E6E0-4B21-862A-DDBD7B224812', '234584C4-2DB1-494F-9434-563A492CD2E9', '4DC676C3-A9E6-428A-BBA0-00D3B4BA241F', '97022FC6-EEF2-46E2-8EDF-FD14DA109466', 'AA4F394E-5E18-450E-ABEC-E45E7C350417', 'D68F312D-BC27-43BC-A9ED-D9ED783E8B36', 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', '3D69FE56-3157-4826-AB50-74B461494603', '103BDFEF-C800-4C4C-9E16-AF05608834B4', 'ECC0BDBA-B14F-4DAE-AA87-AEA5CE026D26', '51BEF171-E2C4-49D5-A293-03625E141B41', '84CCF9C2-928B-42E9-B28C-56059E818044', 'B7EEB9D2-7A57-41BB-A4A3-C59F730DF615', '4E99A4C7-0358-4B6C-BBBB-C87D18F4BB12', 'CE28AB61-0492-4A59-9574-AA9AF276B541', '2273E2B7-C4C8-487C-BCB2-30B7C5E1B89F', '8E183988-EB23-4169-93CB-CBBC35355A5F', '9EF8EAF8-5973-4D12-B1FC-7D710FA6D6BC', '4514D14C-EE3B-4D1A-94DC-F729CBFEFA40', '4939A5A3-0D69-4371-886A-C26434D76ED8', '2E51A705-F0D3-4772-945D-9A18D5EA4310', '6465F86A-70DB-4E2D-ABA8-8DBCD72CAFBB', '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', '58FEB416-06D5-49AA-A463-5A7D48BE2621'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'ChargeDetailID', 'count': 47, 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2'}, {'values': {'label': ['34.0 - 40.4', '40.4 - 46.8', '46.8 - 53.2', '53.2 - 59.6', '59.6 - 66.0', '66.0 - 72.4', '72.4 - 78.8', '78.8 - 85.2', '85.2 - 91.6', '91.6 - 98.0'], 'data': [3, 1, 2, 1, 9, 7, 7, 7, 7, 3]}, 'label': 'Age', 'count': 10, 'min': 34, 'max': 98}, {'values': {'label': ['F', 'M'], 'data': [26, 21]}, 'label': 'Gender', 'count': 2, 'min': 'F', 'max': 'M'}, {'values': {'label': ['2020-01-05 00:00:00', '2020-01-04 00:00:00', '2020-01-03 00:00:00', '2020-01-02 00:00:00'], 'data': [17, 17, 9, 4]}, 'label': 'DateOfEntry', 'count': 4, 'min': '', 'max': ''}, {'values': {'label': ['2020-01-04 00:00:00', '2020-01-03 00:00:00', '2019-12-08 00:00:00', '2019-12-29 00:00:00', '2020-01-02 00:00:00'], 'data': [22, 14, 1, 4, 6]}, 'label': 'DateOfService', 'count': 5, 'min': '', 'max': ''}, {'values': {'label': ['7323A5D3-9249-4F98-9E04-2A53151C8290', '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'A724E957-9B9A-435F-A593-D3272B479E7B', '7A9BF809-317C-435B-9CF4-868B78F71063', '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', '0AC82717-3661-4548-8119-193431DEDEDA', '69294C2D-F7DC-473E-AA9E-53A44DC9B067', '03151B5D-685B-4211-B01F-7BBB1C4A8A49', '530C8C66-2D52-4067-8526-D4F3C94372F0'], 'data': [2, 8, 5, 4, 2, 4, 1, 7, 4, 6, 4]}, 'label': 'ProviderID', 'count': 11, 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3'}, {'values': {'label': ['92014', '92134', '92250', '99204', '92012', '92226', '92225'], 'data': [16, 18, 1, 3, 4, 1, 4]}, 'label': 'CPTCode', 'count': 7, 'min': '92012', 'max': '99204'}, {'values': {'label': ['H33.313', 'H35.3132', 'H43.821', 'E11.3513', 'H35.3122', 'H34.8110', 'H35.3221', 'H34.8310', 'H40.1231', 'H25.89', 'H35.373', 'G45.9', 'E11.3312', 'H35.3211', 'E11.3522', 'E11.9', 'H35.3212', 'H35.3112', 'H35.3210', 'H04.123', 'H33.002', 'H35.352', 'H35.413', 'E11.3413', 'H35.033', 'H35.371', 'H53.9', 'H35.3213', 'E11.3313', 'H33.302', 'H43.813', 'H35.3231', 'H40.1130', 'H33.321', 'H35.712'], 'data': [1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1]}, 'label': 'pdiagnosis1', 'count': 35, 'min': 'E11.3312', 'max': 'H53.9'}, {'values': {'label': ['D31.31', 'H33.313', 'H35.372', 'H35.3132', 'H35.3134', 'H35.3122', 'H35.3221', 'B39.4', 'H35.342', 'H35.373', 'H43.11', 'H35.3211', 'H34.8322', 'H44.23', 'E11.9', 'H35.461', 'H35.052', 'H33.002', 'H25.12', 'H35.352', 'H35.413', 'H35.063', 'E11.3291', 'H35.63', 'H35.033', 'H35.371', 'H35.70', 'H43.12', 'H35.463', 'H43.812', 'E11.3313', 'H43.813', 'Z96.1'], 'data': [2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 3, 1, 1, 1, 2, 1, 1, 1]}, 'label': 'pdiagnosis2', 'count': 33, 'min': 'B39.4', 'max': 'Z96.1'}, {'values': {'label': ['Pulmonary ', 'Orthopedic', 'Cardiology', 'Emergency'], 'data': [20, 3, 5, 19]}, 'label': 'DepartmentCategory', 'count': 4, 'min': 'Cardiology', 'max': 'Pulmonary '}, {'values': {'label': ['Ophtamology'], 'data': [47]}, 'label': 'SpecialityCategory', 'count': 1, 'min': 'Ophtamology', 'max': 'Ophtamology'}, {'values': {'label': ['0.0 - 13.8', '13.8 - 27.6', '27.6 - 41.4', '41.4 - 55.2', '55.2 - 69.0', '69.0 - 82.8', '82.8 - 96.6', '96.6 - 110.4', '110.4 - 124.2', '124.2 - 138.0'], 'data': [12, 12, 12, 6, 1, 2, 0, 1, 0, 1]}, 'label': 'days_to_first_post', 'count': 10, 'min': 0, 'max': 138}, {'values': {'label': ['75F80B5C-AD0E-4A1C-946C-07AEB69B7F9B', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', '2AF763CD-E2CF-4E69-80E5-B00EC12B032F', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'AD96772F-54C7-4750-86B1-1BD565B55244', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '91A3F98C-94F0-4427-8B42-106B6D6C707E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '315C773B-4EF1-4164-AB43-6817D993329F', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'B9F6F179-DD6F-486B-98EE-C207EE60842E', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4'], 'data': [1, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 5, 1, 2, 1, 16, 2, 2, 1, 2]}, 'label': 'PayerId', 'count': 20, 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52'}, {'values': {'label': ['0.0 - 4.6', '4.6 - 9.2', '9.2 - 13.8', '13.8 - 18.4', '18.4 - 23.0', '23.0 - 27.6', '27.6 - 32.2', '32.2 - 36.8', '36.8 - 41.4', '41.4 - 46.0'], 'data': [5, 5, 4, 5, 4, 5, 5, 4, 5, 5]}, 'label': 'ChargeDetailID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 46}, {'values': {'label': ['1', '0'], 'data': [21, 26]}, 'label': 'Gender_stringindexer_transform', 'count': 2, 'min': '0', 'max': '1'}, {'values': {'label': ['3', '5', '4', '2'], 'data': [9, 17, 17, 4]}, 'label': 'DateOfEntry_dayofmonth', 'count': 4, 'min': '2', 'max': '5'}, {'values': {'label': ['1'], 'data': [47]}, 'label': 'DateOfEntry_month', 'count': 1, 'min': '1', 'max': '1'}, {'values': {'label': ['2020'], 'data': [47]}, 'label': 'DateOfEntry_year', 'count': 1, 'min': '2020', 'max': '2020'}, {'values': {'label': ['3', '4', '8', '29', '2'], 'data': [14, 22, 1, 4, 6]}, 'label': 'DateOfService_dayofmonth', 'count': 5, 'min': '2', 'max': '29'}, {'values': {'label': ['12', '1'], 'data': [5, 42]}, 'label': 'DateOfService_month', 'count': 2, 'min': '1', 'max': '12'}, {'values': {'label': ['2019', '2020'], 'data': [5, 42]}, 'label': 'DateOfService_year', 'count': 2, 'min': '2019', 'max': '2020'}, {'values': {'label': ['0.0 - 1.0', '1.0 - 2.0', '2.0 - 3.0', '3.0 - 4.0', '4.0 - 5.0', '5.0 - 6.0', '6.0 - 7.0', '7.0 - 8.0', '8.0 - 9.0', '9.0 - 10.0'], 'data': [8, 7, 6, 5, 4, 4, 4, 4, 2, 3]}, 'label': 'ProviderID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 10}, {'values': {'label': ['0.0', '1.0', '4.0', '3.0', '2.0', '6.0', '5.0'], 'data': [18, 16, 3, 4, 4, 1, 1]}, 'label': 'CPTCode_stringindexer_transform', 'count': 7, 'min': '0.0', 'max': '6.0'}, {'values': {'label': ['0.0 - 3.4', '3.4 - 6.8', '6.8 - 10.2', '10.2 - 13.6', '13.6 - 17.0', '17.0 - 20.4', '20.4 - 23.8', '23.8 - 27.2', '27.2 - 30.6', '30.6 - 34.0'], 'data': [14, 5, 4, 3, 3, 4, 3, 4, 3, 4]}, 'label': 'pdiagnosis1_stringindexer_transform', 'count': 10, 'min': 0, 'max': 34}, {'values': {'label': ['0.0 - 3.2', '3.2 - 6.4', '6.4 - 9.6', '9.6 - 12.8', '12.8 - 16.0', '16.0 - 19.2', '19.2 - 22.4', '22.4 - 25.6', '25.6 - 28.8', '28.8 - 32.0'], 'data': [13, 6, 5, 3, 3, 4, 3, 3, 3, 4]}, 'label': 'pdiagnosis2_stringindexer_transform', 'count': 10, 'min': 0, 'max': 32}, {'values': {'label': ['1', '3', '2', '0'], 'data': [19, 3, 5, 20]}, 'label': 'DepartmentCategory_stringindexer_transform', 'count': 4, 'min': '0', 'max': '3'}, {'values': {'label': ['0'], 'data': [47]}, 'label': 'SpecialityCategory_stringindexer_transform', 'count': 1, 'min': '0', 'max': '0'}, {'values': {'label': ['0.0 - 1.9', '1.9 - 3.8', '3.8 - 5.7', '5.7 - 7.6', '7.6 - 9.5', '9.5 - 11.4', '11.4 - 13.3', '13.3 - 15.2', '15.2 - 17.1', '17.1 - 19.0'], 'data': [21, 6, 4, 4, 2, 2, 2, 2, 2, 2]}, 'label': 'PayerId_stringindexer_transform', 'count': 10, 'min': 0, 'max': 19}], 'FESamplePath': '/FileStore/platform/sampleData/60824e962e36424d53db3377_AutoFE/part-00000-tid-2315973788566987767-9bdba334-e9f6-4cf5-8935-de1b36ce2a7b-482168-1-c000.csv', 'manualfesamplepath': '/FileStore/platform/sampleData/60824e962e36424d53db3377_ManualFE/part-00000-tid-8790912989024766106-9063b124-4e5f-4e0c-aa79-e07bb5fdcc72-482370-1-c000.csv', 'stageId': '60824e962e36424d53db3377', 'stage_id': '60824e962e36424d53db3377', 'status': 1, 'autoFE_flag': True, 'userid': '5ec8228938f3b7a0e925d07e', 'message': 'Successfully executed', 'process_type': 'Auto FE', 'sampledata': [{'ChargeDetailID': '8406E488-7E1E-482B-80B1-4AE30E2F0CD6', 'Age': '71.0', 'Gender': 'F', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-02T00:00:00.000Z', 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92014', 'pdiagnosis1': 'H43.821', 'pdiagnosis2': 'E11.3313', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '9.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'ChargeDetailID_stringindexer_transform': '28.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '2', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '1.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '15.0', 'pdiagnosis2_stringindexer_transform': '20.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '6058CDA2-9A21-4685-BA2F-8DC6AB7F8157', 'Age': '87.0', 'Gender': 'M', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92014', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.371', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '77.0', 'PayerId': '91A3F98C-94F0-4427-8B42-106B6D6C707E', 'ChargeDetailID_stringindexer_transform': '45.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '1.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '12.0'}, {'ChargeDetailID': '5B38B5CA-A8C8-46C1-AFE3-01B32A084701', 'Age': '87.0', 'Gender': 'F', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-02T00:00:00.000Z', 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92014', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'Z96.1', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '78.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '27.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '2', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '0.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '17.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '7152B9FD-20E2-4BAA-9880-ECEEF7B7813A', 'Age': '47.0', 'Gender': 'F', 'DateOfEntry': '2020-01-02T00:00:00.000Z', 'DateOfService': '2019-12-29T00:00:00.000Z', 'ProviderID': '7323A5D3-9249-4F98-9E04-2A53151C8290', 'CPTCode': '92012', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H35.63', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '47.0', 'PayerId': '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'ChargeDetailID_stringindexer_transform': '32.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '2', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '29', 'DateOfService_month': '12', 'DateOfService_year': '2019', 'ProviderID_stringindexer_transform': '8.0', 'CPTCode_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '15.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': '5E8A42D6-AF8A-48A6-A57C-769AA7E36D6A', 'Age': '96.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '7A9BF809-317C-435B-9CF4-868B78F71063', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.813', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '42.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '5.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '25.0', 'pdiagnosis2_stringindexer_transform': '27.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '372F7455-EF3A-428E-923C-3B0465D2F474', 'Age': '62.0', 'Gender': 'M', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92134', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '1.0', 'PayerId': 'B9F6F179-DD6F-486B-98EE-C207EE60842E', 'ChargeDetailID_stringindexer_transform': '40.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '0.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': '2071960F-CC64-4260-948F-3A0FC98E18CE', 'Age': '61.0', 'Gender': 'M', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', 'CPTCode': '99204', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H35.033', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '8.0', 'PayerId': '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'ChargeDetailID_stringindexer_transform': '25.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '9.0', 'CPTCode_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'DepartmentCategory_stringindexer_transform': '3', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': 'BE2AA60D-3002-47AB-A5AD-1FF10FA52D7A', 'Age': '67.0', 'Gender': 'M', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', 'CPTCode': '92225', 'pdiagnosis1': 'H33.313', 'pdiagnosis2': 'H35.373', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '8.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': '44.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '9.0', 'CPTCode_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '33.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'DepartmentCategory_stringindexer_transform': '2', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '50503BED-86A3-4C40-9F05-A8D32F5DABE6', 'Age': '84.0', 'Gender': 'F', 'DateOfEntry': '2020-01-02T00:00:00.000Z', 'DateOfService': '2019-12-29T00:00:00.000Z', 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92225', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H33.313', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '32.0', 'PayerId': '2AF763CD-E2CF-4E69-80E5-B00EC12B032F', 'ChargeDetailID_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '2', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '29', 'DateOfService_month': '12', 'DateOfService_year': '2019', 'ProviderID_stringindexer_transform': '2.0', 'CPTCode_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '32.0', 'DepartmentCategory_stringindexer_transform': '2', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '19.0'}, {'ChargeDetailID': '7ADFBCB9-7E57-44B9-92A5-D9EA6D148794', 'Age': '75.0', 'Gender': 'M', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92014', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'E11.3291', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '35.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '29.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '0.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '132E05B1-87E1-49D5-891D-7D24E6959B94', 'Age': '71.0', 'Gender': 'F', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-02T00:00:00.000Z', 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3221', 'pdiagnosis2': 'H44.23', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '8.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'ChargeDetailID_stringindexer_transform': '36.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '2', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '2.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '30.0', 'pdiagnosis2_stringindexer_transform': '24.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '008CD5CF-D7BE-44C6-AB9A-1EA061BE4800', 'Age': '88.0', 'Gender': 'M', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '7A9BF809-317C-435B-9CF4-868B78F71063', 'CPTCode': '92134', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'E11.3291', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '29.0', 'PayerId': 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'ChargeDetailID_stringindexer_transform': '41.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '5.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '13.0'}, {'ChargeDetailID': 'A784EB62-A0F5-463D-BD9F-412AF036DDD7', 'Age': '91.0', 'Gender': 'M', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3213', 'pdiagnosis2': 'H35.352', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '33.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '19.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '3.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '26.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C481E307-0A94-467B-BF82-53D34728427E', 'Age': '69.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2019-12-08T00:00:00.000Z', 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92250', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.371', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '16.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'ChargeDetailID_stringindexer_transform': '21.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '8', 'DateOfService_month': '12', 'DateOfService_year': '2019', 'ProviderID_stringindexer_transform': '0.0', 'CPTCode_stringindexer_transform': '5.0', 'pdiagnosis1_stringindexer_transform': '22.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '43846138-A55C-4877-863F-C1AC659A3F44', 'Age': '57.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92134', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.063', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '62.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '2.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '80CB21C5-3FFF-4F35-B228-4A1ABB4D2015', 'Age': '78.0', 'Gender': 'M', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '7A9BF809-317C-435B-9CF4-868B78F71063', 'CPTCode': '92134', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.033', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '42.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '23.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '5.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '6BDD5CED-1DE1-4774-BC24-03127487C3D3', 'Age': '73.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '69294C2D-F7DC-473E-AA9E-53A44DC9B067', 'CPTCode': '92134', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H43.812', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '40.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '7.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '19.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '4D15A79D-1B97-461B-8994-C9DB9BBFE5FF', 'Age': '76.0', 'Gender': 'M', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92014', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.033', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '25.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '24.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '3.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '32.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'DB8DA258-E24D-43A0-BE20-9DABC58150D6', 'Age': '85.0', 'Gender': 'M', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92012', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '18.0', 'PayerId': '193BA5E7-F797-4FD1-A9AB-27590E630FCB', 'ChargeDetailID_stringindexer_transform': '16.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '0.0', 'CPTCode_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '18.0'}, {'ChargeDetailID': '327659EE-D609-4ED5-83FB-A87DF7EAC224', 'Age': '65.0', 'Gender': 'M', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92134', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H35.413', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '15.0', 'PayerId': '315C773B-4EF1-4164-AB43-6817D993329F', 'ChargeDetailID_stringindexer_transform': '42.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '0.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '18.0', 'pdiagnosis2_stringindexer_transform': '18.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '15.0'}, {'ChargeDetailID': '1E6165E2-7C9B-4680-972C-72BFD783CCA6', 'Age': '79.0', 'Gender': 'M', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CPTCode': '92014', 'pdiagnosis1': 'E11.9', 'pdiagnosis2': 'H35.033', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '36.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '13.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '4.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A21E71B8-5267-4260-B174-EBEACF6765A6', 'Age': '80.0', 'Gender': 'F', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-02T00:00:00.000Z', 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '43.0', 'PayerId': 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'ChargeDetailID_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '2', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '1.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '8.0'}, {'ChargeDetailID': '04027CE6-725D-45D3-B3F5-FE1684208F6F', 'Age': '39.0', 'Gender': 'F', 'DateOfEntry': '2020-01-02T00:00:00.000Z', 'DateOfService': '2019-12-29T00:00:00.000Z', 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '99204', 'pdiagnosis1': 'H35.413', 'pdiagnosis2': 'H35.463', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '47.0', 'PayerId': '75F80B5C-AD0E-4A1C-946C-07AEB69B7F9B', 'ChargeDetailID_stringindexer_transform': '15.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '2', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '29', 'DateOfService_month': '12', 'DateOfService_year': '2019', 'ProviderID_stringindexer_transform': '2.0', 'CPTCode_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '16.0', 'pdiagnosis2_stringindexer_transform': '16.0', 'DepartmentCategory_stringindexer_transform': '3', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '11.0'}, {'ChargeDetailID': 'BB460B17-E6E0-4B21-862A-DDBD7B224812', 'Age': '34.0', 'Gender': 'M', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-02T00:00:00.000Z', 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92014', 'pdiagnosis1': 'H25.89', 'pdiagnosis2': 'H33.002', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '10.0', 'PayerId': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'ChargeDetailID_stringindexer_transform': '46.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '2', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '1.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '23.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '10.0'}, {'ChargeDetailID': '234584C4-2DB1-494F-9434-563A492CD2E9', 'Age': '82.0', 'Gender': 'M', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '69294C2D-F7DC-473E-AA9E-53A44DC9B067', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '36.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '17.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '7.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '4DC676C3-A9E6-428A-BBA0-00D3B4BA241F', 'Age': '76.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '69294C2D-F7DC-473E-AA9E-53A44DC9B067', 'CPTCode': '92014', 'pdiagnosis1': 'H04.123', 'pdiagnosis2': 'H35.3211', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '13.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'ChargeDetailID_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '7.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '17.0', 'pdiagnosis2_stringindexer_transform': '31.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '97022FC6-EEF2-46E2-8EDF-FD14DA109466', 'Age': '69.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '69294C2D-F7DC-473E-AA9E-53A44DC9B067', 'CPTCode': '92134', 'pdiagnosis1': 'H35.352', 'pdiagnosis2': 'H35.372', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '138.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '35.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '7.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '31.0', 'pdiagnosis2_stringindexer_transform': '13.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'AA4F394E-5E18-450E-ABEC-E45E7C350417', 'Age': '77.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'E11.9', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '26.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '33.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '4.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D68F312D-BC27-43BC-A9ED-D9ED783E8B36', 'Age': '65.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92012', 'pdiagnosis1': 'G45.9', 'pdiagnosis2': 'D31.31', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '14.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'ChargeDetailID_stringindexer_transform': '43.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '0.0', 'CPTCode_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '27.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', 'Age': '89.0', 'Gender': 'M', 'DateOfEntry': '2020-01-03T00:00:00.000Z', 'DateOfService': '2020-01-02T00:00:00.000Z', 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92134', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H34.8322', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '15.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'ChargeDetailID_stringindexer_transform': '26.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '3', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '2', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '2.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '28.0', 'pdiagnosis2_stringindexer_transform': '21.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '3D69FE56-3157-4826-AB50-74B461494603', 'Age': '96.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92014', 'pdiagnosis1': 'H35.3210', 'pdiagnosis2': 'E11.9', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '32.0', 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'ChargeDetailID_stringindexer_transform': '37.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '1.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '29.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '14.0'}, {'ChargeDetailID': '103BDFEF-C800-4C4C-9E16-AF05608834B4', 'Age': '84.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CPTCode': '92014', 'pdiagnosis1': 'H40.1130', 'pdiagnosis2': 'H35.3132', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '28.0', 'PayerId': 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'ChargeDetailID_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '4.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '21.0', 'pdiagnosis2_stringindexer_transform': '22.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '17.0'}, {'ChargeDetailID': 'ECC0BDBA-B14F-4DAE-AA87-AEA5CE026D26', 'Age': '80.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CPTCode': '92014', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H25.12', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '26.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '20.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '4.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '19.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '51BEF171-E2C4-49D5-A293-03625E141B41', 'Age': '87.0', 'Gender': 'M', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92225', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.342', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '16.0', 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'ChargeDetailID_stringindexer_transform': '22.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '3.0', 'CPTCode_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '29.0', 'DepartmentCategory_stringindexer_transform': '2', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '16.0'}, {'ChargeDetailID': '84CCF9C2-928B-42E9-B28C-56059E818044', 'Age': '98.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '36.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '1.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '25.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'B7EEB9D2-7A57-41BB-A4A3-C59F730DF615', 'Age': '64.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'CPTCode': '92014', 'pdiagnosis1': 'H33.321', 'pdiagnosis2': 'H35.461', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '9.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': '14.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '10.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '20.0', 'pdiagnosis2_stringindexer_transform': '30.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '4E99A4C7-0358-4B6C-BBBB-C87D18F4BB12', 'Age': '89.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92225', 'pdiagnosis1': 'H53.9', 'pdiagnosis2': 'B39.4', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '21.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '18.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '1.0', 'CPTCode_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '23.0', 'DepartmentCategory_stringindexer_transform': '2', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'CE28AB61-0492-4A59-9574-AA9AF276B541', 'Age': '63.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92014', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.12', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '99.0', 'PayerId': '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'ChargeDetailID_stringindexer_transform': '38.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '3.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '14.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '2273E2B7-C4C8-487C-BCB2-30B7C5E1B89F', 'Age': '72.0', 'Gender': 'F', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '530C8C66-2D52-4067-8526-D4F3C94372F0', 'CPTCode': '92134', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.371', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '37.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '34.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '6.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '8E183988-EB23-4169-93CB-CBBC35355A5F', 'Age': '76.0', 'Gender': 'M', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3134', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '29.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '3.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9EF8EAF8-5973-4D12-B1FC-7D710FA6D6BC', 'Age': '48.0', 'Gender': 'F', 'DateOfEntry': '2020-01-02T00:00:00.000Z', 'DateOfService': '2019-12-29T00:00:00.000Z', 'ProviderID': '7323A5D3-9249-4F98-9E04-2A53151C8290', 'CPTCode': '92012', 'pdiagnosis1': 'E11.3522', 'pdiagnosis2': 'H43.11', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '13.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '2', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '29', 'DateOfService_month': '12', 'DateOfService_year': '2019', 'ProviderID_stringindexer_transform': '8.0', 'CPTCode_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '26.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '4514D14C-EE3B-4D1A-94DC-F729CBFEFA40', 'Age': '44.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '530C8C66-2D52-4067-8526-D4F3C94372F0', 'CPTCode': '92014', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.052', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '0.0', 'PayerId': 'B9F6F179-DD6F-486B-98EE-C207EE60842E', 'ChargeDetailID_stringindexer_transform': '30.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '6.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': '4939A5A3-0D69-4371-886A-C26434D76ED8', 'Age': '39.0', 'Gender': 'M', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92014', 'pdiagnosis1': 'H35.712', 'pdiagnosis2': 'H35.70', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '9.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'ChargeDetailID_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '2.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '28.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '2E51A705-F0D3-4772-945D-9A18D5EA4310', 'Age': '60.0', 'Gender': 'M', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '530C8C66-2D52-4067-8526-D4F3C94372F0', 'CPTCode': '92014', 'pdiagnosis1': 'H40.1231', 'pdiagnosis2': 'D31.31', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '14.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'ChargeDetailID_stringindexer_transform': '31.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '6.0', 'CPTCode_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '24.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'DepartmentCategory_stringindexer_transform': '0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '9.0'}, {'ChargeDetailID': '6465F86A-70DB-4E2D-ABA8-8DBCD72CAFBB', 'Age': '61.0', 'Gender': 'M', 'DateOfEntry': '2020-01-04T00:00:00.000Z', 'DateOfService': '2020-01-03T00:00:00.000Z', 'ProviderID': '7A9BF809-317C-435B-9CF4-868B78F71063', 'CPTCode': '99204', 'pdiagnosis1': 'H33.302', 'pdiagnosis2': 'H43.812', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '8.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '4', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '3', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '5.0', 'CPTCode_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '34.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'DepartmentCategory_stringindexer_transform': '3', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'Age': '65.0', 'Gender': 'F', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92134', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.373', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '14.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'ChargeDetailID_stringindexer_transform': '39.0', 'Gender_stringindexer_transform': '0', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '0.0', 'CPTCode_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'DepartmentCategory_stringindexer_transform': '1', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '58FEB416-06D5-49AA-A463-5A7D48BE2621', 'Age': '70.0', 'Gender': 'M', 'DateOfEntry': '2020-01-05T00:00:00.000Z', 'DateOfService': '2020-01-04T00:00:00.000Z', 'ProviderID': '530C8C66-2D52-4067-8526-D4F3C94372F0', 'CPTCode': '92226', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': '53.0', 'PayerId': '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'ChargeDetailID_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '1', 'DateOfEntry_dayofmonth': '5', 'DateOfEntry_month': '1', 'DateOfEntry_year': '2020', 'DateOfService_dayofmonth': '4', 'DateOfService_month': '1', 'DateOfService_year': '2020', 'ProviderID_stringindexer_transform': '6.0', 'CPTCode_stringindexer_transform': '6.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'DepartmentCategory_stringindexer_transform': '2', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': ''}], 'attributesFT': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '71.83', 'stddev': '15.61', 'min': '34', 'max': '98', 'missing': '0'}}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False}, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False}, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '92546.21', 'stddev': '1758.91', 'min': '92012', 'max': '99204', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'E11.3312', 'max': 'H53.9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'B39.4', 'max': 'Z96.1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'days_to_first_post', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '30.79', 'stddev': '26.24', 'min': '0', 'max': '138', 'missing': '0'}}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '23.0', 'stddev': '13.71', 'min': '0.0', 'max': '46.0', 'missing': '0'}}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}}, {'feature': 'DateOfEntry_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '4.0', 'stddev': '0.96', 'min': '2', 'max': '5', 'missing': '0'}}, {'feature': 'DateOfEntry_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '1.0', 'stddev': '0.0', 'min': '1', 'max': '1', 'missing': '0'}}, {'feature': 'DateOfEntry_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}}, {'feature': 'DateOfService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '5.66', 'stddev': '7.26', 'min': '2', 'max': '29', 'missing': '0'}}, {'feature': 'DateOfService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2.17', 'stddev': '3.43', 'min': '1', 'max': '12', 'missing': '0'}}, {'feature': 'DateOfService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2019.89', 'stddev': '0.31', 'min': '2019', 'max': '2020', 'missing': '0'}}, {'feature': 'ProviderID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '3.53', 'stddev': '2.89', 'min': '0.0', 'max': '10.0', 'missing': '0'}}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '1.26', 'stddev': '1.5', 'min': '0.0', 'max': '6.0', 'missing': '0'}}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '13.06', 'stddev': '11.16', 'min': '0.0', 'max': '34.0', 'missing': '0'}}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '12.13', 'stddev': '10.16', 'min': '0.0', 'max': '32.0', 'missing': '0'}}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.81', 'stddev': '0.88', 'min': '0', 'max': '3', 'missing': '0'}}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '4.79', 'stddev': '5.79', 'min': '0.0', 'max': '19.0', 'missing': '0'}}], 'path_AutoSample': '/FileStore/platform/sampleData/60824e962e36424d53db3377_AutoFE/part-00000-tid-2315973788566987767-9bdba334-e9f6-4cf5-8935-de1b36ce2a7b-482168-1-c000.csv', 'listOfGraph': [{'values': {'label': ['8406E488-7E1E-482B-80B1-4AE30E2F0CD6', '6058CDA2-9A21-4685-BA2F-8DC6AB7F8157', '5B38B5CA-A8C8-46C1-AFE3-01B32A084701', '7152B9FD-20E2-4BAA-9880-ECEEF7B7813A', '5E8A42D6-AF8A-48A6-A57C-769AA7E36D6A', '372F7455-EF3A-428E-923C-3B0465D2F474', '2071960F-CC64-4260-948F-3A0FC98E18CE', 'BE2AA60D-3002-47AB-A5AD-1FF10FA52D7A', '50503BED-86A3-4C40-9F05-A8D32F5DABE6', '7ADFBCB9-7E57-44B9-92A5-D9EA6D148794', '132E05B1-87E1-49D5-891D-7D24E6959B94', '008CD5CF-D7BE-44C6-AB9A-1EA061BE4800', 'A784EB62-A0F5-463D-BD9F-412AF036DDD7', 'C481E307-0A94-467B-BF82-53D34728427E', '43846138-A55C-4877-863F-C1AC659A3F44', '80CB21C5-3FFF-4F35-B228-4A1ABB4D2015', '6BDD5CED-1DE1-4774-BC24-03127487C3D3', '4D15A79D-1B97-461B-8994-C9DB9BBFE5FF', 'DB8DA258-E24D-43A0-BE20-9DABC58150D6', '327659EE-D609-4ED5-83FB-A87DF7EAC224', '1E6165E2-7C9B-4680-972C-72BFD783CCA6', 'A21E71B8-5267-4260-B174-EBEACF6765A6', '04027CE6-725D-45D3-B3F5-FE1684208F6F', 'BB460B17-E6E0-4B21-862A-DDBD7B224812', '234584C4-2DB1-494F-9434-563A492CD2E9', '4DC676C3-A9E6-428A-BBA0-00D3B4BA241F', '97022FC6-EEF2-46E2-8EDF-FD14DA109466', 'AA4F394E-5E18-450E-ABEC-E45E7C350417', 'D68F312D-BC27-43BC-A9ED-D9ED783E8B36', 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', '3D69FE56-3157-4826-AB50-74B461494603', '103BDFEF-C800-4C4C-9E16-AF05608834B4', 'ECC0BDBA-B14F-4DAE-AA87-AEA5CE026D26', '51BEF171-E2C4-49D5-A293-03625E141B41', '84CCF9C2-928B-42E9-B28C-56059E818044', 'B7EEB9D2-7A57-41BB-A4A3-C59F730DF615', '4E99A4C7-0358-4B6C-BBBB-C87D18F4BB12', 'CE28AB61-0492-4A59-9574-AA9AF276B541', '2273E2B7-C4C8-487C-BCB2-30B7C5E1B89F', '8E183988-EB23-4169-93CB-CBBC35355A5F', '9EF8EAF8-5973-4D12-B1FC-7D710FA6D6BC', '4514D14C-EE3B-4D1A-94DC-F729CBFEFA40', '4939A5A3-0D69-4371-886A-C26434D76ED8', '2E51A705-F0D3-4772-945D-9A18D5EA4310', '6465F86A-70DB-4E2D-ABA8-8DBCD72CAFBB', '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', '58FEB416-06D5-49AA-A463-5A7D48BE2621'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'ChargeDetailID', 'count': 47, 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2'}, {'values': {'label': ['34.0 - 40.4', '40.4 - 46.8', '46.8 - 53.2', '53.2 - 59.6', '59.6 - 66.0', '66.0 - 72.4', '72.4 - 78.8', '78.8 - 85.2', '85.2 - 91.6', '91.6 - 98.0'], 'data': [3, 1, 2, 1, 9, 7, 7, 7, 7, 3]}, 'label': 'Age', 'count': 10, 'min': 34, 'max': 98}, {'values': {'label': ['F', 'M'], 'data': [26, 21]}, 'label': 'Gender', 'count': 2, 'min': 'F', 'max': 'M'}, {'values': {'label': ['2020-01-05 00:00:00', '2020-01-04 00:00:00', '2020-01-03 00:00:00', '2020-01-02 00:00:00'], 'data': [17, 17, 9, 4]}, 'label': 'DateOfEntry', 'count': 4, 'min': '', 'max': ''}, {'values': {'label': ['2020-01-04 00:00:00', '2020-01-03 00:00:00', '2019-12-08 00:00:00', '2019-12-29 00:00:00', '2020-01-02 00:00:00'], 'data': [22, 14, 1, 4, 6]}, 'label': 'DateOfService', 'count': 5, 'min': '', 'max': ''}, {'values': {'label': ['7323A5D3-9249-4F98-9E04-2A53151C8290', '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'A724E957-9B9A-435F-A593-D3272B479E7B', '7A9BF809-317C-435B-9CF4-868B78F71063', '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', '0AC82717-3661-4548-8119-193431DEDEDA', '69294C2D-F7DC-473E-AA9E-53A44DC9B067', '03151B5D-685B-4211-B01F-7BBB1C4A8A49', '530C8C66-2D52-4067-8526-D4F3C94372F0'], 'data': [2, 8, 5, 4, 2, 4, 1, 7, 4, 6, 4]}, 'label': 'ProviderID', 'count': 11, 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3'}, {'values': {'label': ['92014', '92134', '92250', '99204', '92012', '92226', '92225'], 'data': [16, 18, 1, 3, 4, 1, 4]}, 'label': 'CPTCode', 'count': 7, 'min': '92012', 'max': '99204'}, {'values': {'label': ['H33.313', 'H35.3132', 'H43.821', 'E11.3513', 'H35.3122', 'H34.8110', 'H35.3221', 'H34.8310', 'H40.1231', 'H25.89', 'H35.373', 'G45.9', 'E11.3312', 'H35.3211', 'E11.3522', 'E11.9', 'H35.3212', 'H35.3112', 'H35.3210', 'H04.123', 'H33.002', 'H35.352', 'H35.413', 'E11.3413', 'H35.033', 'H35.371', 'H53.9', 'H35.3213', 'E11.3313', 'H33.302', 'H43.813', 'H35.3231', 'H40.1130', 'H33.321', 'H35.712'], 'data': [1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1]}, 'label': 'pdiagnosis1', 'count': 35, 'min': 'E11.3312', 'max': 'H53.9'}, {'values': {'label': ['D31.31', 'H33.313', 'H35.372', 'H35.3132', 'H35.3134', 'H35.3122', 'H35.3221', 'B39.4', 'H35.342', 'H35.373', 'H43.11', 'H35.3211', 'H34.8322', 'H44.23', 'E11.9', 'H35.461', 'H35.052', 'H33.002', 'H25.12', 'H35.352', 'H35.413', 'H35.063', 'E11.3291', 'H35.63', 'H35.033', 'H35.371', 'H35.70', 'H43.12', 'H35.463', 'H43.812', 'E11.3313', 'H43.813', 'Z96.1'], 'data': [2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 3, 1, 1, 1, 2, 1, 1, 1]}, 'label': 'pdiagnosis2', 'count': 33, 'min': 'B39.4', 'max': 'Z96.1'}, {'values': {'label': ['Pulmonary ', 'Orthopedic', 'Cardiology', 'Emergency'], 'data': [20, 3, 5, 19]}, 'label': 'DepartmentCategory', 'count': 4, 'min': 'Cardiology', 'max': 'Pulmonary '}, {'values': {'label': ['Ophtamology'], 'data': [47]}, 'label': 'SpecialityCategory', 'count': 1, 'min': 'Ophtamology', 'max': 'Ophtamology'}, {'values': {'label': ['0.0 - 13.8', '13.8 - 27.6', '27.6 - 41.4', '41.4 - 55.2', '55.2 - 69.0', '69.0 - 82.8', '82.8 - 96.6', '96.6 - 110.4', '110.4 - 124.2', '124.2 - 138.0'], 'data': [12, 12, 12, 6, 1, 2, 0, 1, 0, 1]}, 'label': 'days_to_first_post', 'count': 10, 'min': 0, 'max': 138}, {'values': {'label': ['75F80B5C-AD0E-4A1C-946C-07AEB69B7F9B', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', '2AF763CD-E2CF-4E69-80E5-B00EC12B032F', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'AD96772F-54C7-4750-86B1-1BD565B55244', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '91A3F98C-94F0-4427-8B42-106B6D6C707E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '315C773B-4EF1-4164-AB43-6817D993329F', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'B9F6F179-DD6F-486B-98EE-C207EE60842E', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4'], 'data': [1, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 5, 1, 2, 1, 16, 2, 2, 1, 2]}, 'label': 'PayerId', 'count': 20, 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52'}, {'values': {'label': ['0.0 - 4.6', '4.6 - 9.2', '9.2 - 13.8', '13.8 - 18.4', '18.4 - 23.0', '23.0 - 27.6', '27.6 - 32.2', '32.2 - 36.8', '36.8 - 41.4', '41.4 - 46.0'], 'data': [5, 5, 4, 5, 4, 5, 5, 4, 5, 5]}, 'label': 'ChargeDetailID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 46}, {'values': {'label': ['1', '0'], 'data': [21, 26]}, 'label': 'Gender_stringindexer_transform', 'count': 2, 'min': '0', 'max': '1'}, {'values': {'label': ['3', '5', '4', '2'], 'data': [9, 17, 17, 4]}, 'label': 'DateOfEntry_dayofmonth', 'count': 4, 'min': '2', 'max': '5'}, {'values': {'label': ['1'], 'data': [47]}, 'label': 'DateOfEntry_month', 'count': 1, 'min': '1', 'max': '1'}, {'values': {'label': ['2020'], 'data': [47]}, 'label': 'DateOfEntry_year', 'count': 1, 'min': '2020', 'max': '2020'}, {'values': {'label': ['3', '4', '8', '29', '2'], 'data': [14, 22, 1, 4, 6]}, 'label': 'DateOfService_dayofmonth', 'count': 5, 'min': '2', 'max': '29'}, {'values': {'label': ['12', '1'], 'data': [5, 42]}, 'label': 'DateOfService_month', 'count': 2, 'min': '1', 'max': '12'}, {'values': {'label': ['2019', '2020'], 'data': [5, 42]}, 'label': 'DateOfService_year', 'count': 2, 'min': '2019', 'max': '2020'}, {'values': {'label': ['0.0 - 1.0', '1.0 - 2.0', '2.0 - 3.0', '3.0 - 4.0', '4.0 - 5.0', '5.0 - 6.0', '6.0 - 7.0', '7.0 - 8.0', '8.0 - 9.0', '9.0 - 10.0'], 'data': [8, 7, 6, 5, 4, 4, 4, 4, 2, 3]}, 'label': 'ProviderID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 10}, {'values': {'label': ['0.0', '1.0', '4.0', '3.0', '2.0', '6.0', '5.0'], 'data': [18, 16, 3, 4, 4, 1, 1]}, 'label': 'CPTCode_stringindexer_transform', 'count': 7, 'min': '0.0', 'max': '6.0'}, {'values': {'label': ['0.0 - 3.4', '3.4 - 6.8', '6.8 - 10.2', '10.2 - 13.6', '13.6 - 17.0', '17.0 - 20.4', '20.4 - 23.8', '23.8 - 27.2', '27.2 - 30.6', '30.6 - 34.0'], 'data': [14, 5, 4, 3, 3, 4, 3, 4, 3, 4]}, 'label': 'pdiagnosis1_stringindexer_transform', 'count': 10, 'min': 0, 'max': 34}, {'values': {'label': ['0.0 - 3.2', '3.2 - 6.4', '6.4 - 9.6', '9.6 - 12.8', '12.8 - 16.0', '16.0 - 19.2', '19.2 - 22.4', '22.4 - 25.6', '25.6 - 28.8', '28.8 - 32.0'], 'data': [13, 6, 5, 3, 3, 4, 3, 3, 3, 4]}, 'label': 'pdiagnosis2_stringindexer_transform', 'count': 10, 'min': 0, 'max': 32}, {'values': {'label': ['1', '3', '2', '0'], 'data': [19, 3, 5, 20]}, 'label': 'DepartmentCategory_stringindexer_transform', 'count': 4, 'min': '0', 'max': '3'}, {'values': {'label': ['0'], 'data': [47]}, 'label': 'SpecialityCategory_stringindexer_transform', 'count': 1, 'min': '0', 'max': '0'}, {'values': {'label': ['0.0 - 1.9', '1.9 - 3.8', '3.8 - 5.7', '5.7 - 7.6', '7.6 - 9.5', '9.5 - 11.4', '11.4 - 13.3', '13.3 - 15.2', '15.2 - 17.1', '17.1 - 19.0'], 'data': [21, 6, 4, 4, 2, 2, 2, 2, 2, 2]}, 'label': 'PayerId_stringindexer_transform', 'count': 10, 'min': 0, 'max': 19}], 'path_ManualSample': '/FileStore/platform/sampleData/60824e962e36424d53db3377_ManualFE/part-00000-tid-8790912989024766106-9063b124-4e5f-4e0c-aa79-e07bb5fdcc72-482370-1-c000.csv', 'graphdata': {'ChargeDetailID': {'label': ['8406E488-7E1E-482B-80B1-4AE30E2F0CD6', '6058CDA2-9A21-4685-BA2F-8DC6AB7F8157', '5B38B5CA-A8C8-46C1-AFE3-01B32A084701', '7152B9FD-20E2-4BAA-9880-ECEEF7B7813A', '5E8A42D6-AF8A-48A6-A57C-769AA7E36D6A', '372F7455-EF3A-428E-923C-3B0465D2F474', '2071960F-CC64-4260-948F-3A0FC98E18CE', 'BE2AA60D-3002-47AB-A5AD-1FF10FA52D7A', '50503BED-86A3-4C40-9F05-A8D32F5DABE6', '7ADFBCB9-7E57-44B9-92A5-D9EA6D148794', '132E05B1-87E1-49D5-891D-7D24E6959B94', '008CD5CF-D7BE-44C6-AB9A-1EA061BE4800', 'A784EB62-A0F5-463D-BD9F-412AF036DDD7', 'C481E307-0A94-467B-BF82-53D34728427E', '43846138-A55C-4877-863F-C1AC659A3F44', '80CB21C5-3FFF-4F35-B228-4A1ABB4D2015', '6BDD5CED-1DE1-4774-BC24-03127487C3D3', '4D15A79D-1B97-461B-8994-C9DB9BBFE5FF', 'DB8DA258-E24D-43A0-BE20-9DABC58150D6', '327659EE-D609-4ED5-83FB-A87DF7EAC224', '1E6165E2-7C9B-4680-972C-72BFD783CCA6', 'A21E71B8-5267-4260-B174-EBEACF6765A6', '04027CE6-725D-45D3-B3F5-FE1684208F6F', 'BB460B17-E6E0-4B21-862A-DDBD7B224812', '234584C4-2DB1-494F-9434-563A492CD2E9', '4DC676C3-A9E6-428A-BBA0-00D3B4BA241F', '97022FC6-EEF2-46E2-8EDF-FD14DA109466', 'AA4F394E-5E18-450E-ABEC-E45E7C350417', 'D68F312D-BC27-43BC-A9ED-D9ED783E8B36', 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', '3D69FE56-3157-4826-AB50-74B461494603', '103BDFEF-C800-4C4C-9E16-AF05608834B4', 'ECC0BDBA-B14F-4DAE-AA87-AEA5CE026D26', '51BEF171-E2C4-49D5-A293-03625E141B41', '84CCF9C2-928B-42E9-B28C-56059E818044', 'B7EEB9D2-7A57-41BB-A4A3-C59F730DF615', '4E99A4C7-0358-4B6C-BBBB-C87D18F4BB12', 'CE28AB61-0492-4A59-9574-AA9AF276B541', '2273E2B7-C4C8-487C-BCB2-30B7C5E1B89F', '8E183988-EB23-4169-93CB-CBBC35355A5F', '9EF8EAF8-5973-4D12-B1FC-7D710FA6D6BC', '4514D14C-EE3B-4D1A-94DC-F729CBFEFA40', '4939A5A3-0D69-4371-886A-C26434D76ED8', '2E51A705-F0D3-4772-945D-9A18D5EA4310', '6465F86A-70DB-4E2D-ABA8-8DBCD72CAFBB', '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', '58FEB416-06D5-49AA-A463-5A7D48BE2621'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'Age': {'label': ['34.0 - 40.4', '40.4 - 46.8', '46.8 - 53.2', '53.2 - 59.6', '59.6 - 66.0', '66.0 - 72.4', '72.4 - 78.8', '78.8 - 85.2', '85.2 - 91.6', '91.6 - 98.0'], 'data': [3, 1, 2, 1, 9, 7, 7, 7, 7, 3]}, 'Gender': {'label': ['F', 'M'], 'data': [26, 21]}, 'DateOfEntry': {'label': ['2020-01-05 00:00:00', '2020-01-04 00:00:00', '2020-01-03 00:00:00', '2020-01-02 00:00:00'], 'data': [17, 17, 9, 4]}, 'DateOfService': {'label': ['2020-01-04 00:00:00', '2020-01-03 00:00:00', '2019-12-08 00:00:00', '2019-12-29 00:00:00', '2020-01-02 00:00:00'], 'data': [22, 14, 1, 4, 6]}, 'ProviderID': {'label': ['7323A5D3-9249-4F98-9E04-2A53151C8290', '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'A724E957-9B9A-435F-A593-D3272B479E7B', '7A9BF809-317C-435B-9CF4-868B78F71063', '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', '0AC82717-3661-4548-8119-193431DEDEDA', '69294C2D-F7DC-473E-AA9E-53A44DC9B067', '03151B5D-685B-4211-B01F-7BBB1C4A8A49', '530C8C66-2D52-4067-8526-D4F3C94372F0'], 'data': [2, 8, 5, 4, 2, 4, 1, 7, 4, 6, 4]}, 'CPTCode': {'label': ['92014', '92134', '92250', '99204', '92012', '92226', '92225'], 'data': [16, 18, 1, 3, 4, 1, 4]}, 'pdiagnosis1': {'label': ['H33.313', 'H35.3132', 'H43.821', 'E11.3513', 'H35.3122', 'H34.8110', 'H35.3221', 'H34.8310', 'H40.1231', 'H25.89', 'H35.373', 'G45.9', 'E11.3312', 'H35.3211', 'E11.3522', 'E11.9', 'H35.3212', 'H35.3112', 'H35.3210', 'H04.123', 'H33.002', 'H35.352', 'H35.413', 'E11.3413', 'H35.033', 'H35.371', 'H53.9', 'H35.3213', 'E11.3313', 'H33.302', 'H43.813', 'H35.3231', 'H40.1130', 'H33.321', 'H35.712'], 'data': [1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1]}, 'pdiagnosis2': {'label': ['D31.31', 'H33.313', 'H35.372', 'H35.3132', 'H35.3134', 'H35.3122', 'H35.3221', 'B39.4', 'H35.342', 'H35.373', 'H43.11', 'H35.3211', 'H34.8322', 'H44.23', 'E11.9', 'H35.461', 'H35.052', 'H33.002', 'H25.12', 'H35.352', 'H35.413', 'H35.063', 'E11.3291', 'H35.63', 'H35.033', 'H35.371', 'H35.70', 'H43.12', 'H35.463', 'H43.812', 'E11.3313', 'H43.813', 'Z96.1'], 'data': [2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 3, 1, 1, 1, 2, 1, 1, 1]}, 'DepartmentCategory': {'label': ['Pulmonary ', 'Orthopedic', 'Cardiology', 'Emergency'], 'data': [20, 3, 5, 19]}, 'SpecialityCategory': {'label': ['Ophtamology'], 'data': [47]}, 'days_to_first_post': {'label': ['0.0 - 13.8', '13.8 - 27.6', '27.6 - 41.4', '41.4 - 55.2', '55.2 - 69.0', '69.0 - 82.8', '82.8 - 96.6', '96.6 - 110.4', '110.4 - 124.2', '124.2 - 138.0'], 'data': [12, 12, 12, 6, 1, 2, 0, 1, 0, 1]}, 'PayerId': {'label': ['75F80B5C-AD0E-4A1C-946C-07AEB69B7F9B', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', '2AF763CD-E2CF-4E69-80E5-B00EC12B032F', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'AD96772F-54C7-4750-86B1-1BD565B55244', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '91A3F98C-94F0-4427-8B42-106B6D6C707E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '315C773B-4EF1-4164-AB43-6817D993329F', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'B9F6F179-DD6F-486B-98EE-C207EE60842E', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4'], 'data': [1, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 5, 1, 2, 1, 16, 2, 2, 1, 2]}, 'ChargeDetailID_stringindexer_transform': {'label': ['0.0 - 4.6', '4.6 - 9.2', '9.2 - 13.8', '13.8 - 18.4', '18.4 - 23.0', '23.0 - 27.6', '27.6 - 32.2', '32.2 - 36.8', '36.8 - 41.4', '41.4 - 46.0'], 'data': [5, 5, 4, 5, 4, 5, 5, 4, 5, 5]}, 'Gender_stringindexer_transform': {'label': ['1', '0'], 'data': [21, 26]}, 'DateOfEntry_dayofmonth': {'label': ['3', '5', '4', '2'], 'data': [9, 17, 17, 4]}, 'DateOfEntry_month': {'label': ['1'], 'data': [47]}, 'DateOfEntry_year': {'label': ['2020'], 'data': [47]}, 'DateOfService_dayofmonth': {'label': ['3', '4', '8', '29', '2'], 'data': [14, 22, 1, 4, 6]}, 'DateOfService_month': {'label': ['12', '1'], 'data': [5, 42]}, 'DateOfService_year': {'label': ['2019', '2020'], 'data': [5, 42]}, 'ProviderID_stringindexer_transform': {'label': ['0.0 - 1.0', '1.0 - 2.0', '2.0 - 3.0', '3.0 - 4.0', '4.0 - 5.0', '5.0 - 6.0', '6.0 - 7.0', '7.0 - 8.0', '8.0 - 9.0', '9.0 - 10.0'], 'data': [8, 7, 6, 5, 4, 4, 4, 4, 2, 3]}, 'CPTCode_stringindexer_transform': {'label': ['0.0', '1.0', '4.0', '3.0', '2.0', '6.0', '5.0'], 'data': [18, 16, 3, 4, 4, 1, 1]}, 'pdiagnosis1_stringindexer_transform': {'label': ['0.0 - 3.4', '3.4 - 6.8', '6.8 - 10.2', '10.2 - 13.6', '13.6 - 17.0', '17.0 - 20.4', '20.4 - 23.8', '23.8 - 27.2', '27.2 - 30.6', '30.6 - 34.0'], 'data': [14, 5, 4, 3, 3, 4, 3, 4, 3, 4]}, 'pdiagnosis2_stringindexer_transform': {'label': ['0.0 - 3.2', '3.2 - 6.4', '6.4 - 9.6', '9.6 - 12.8', '12.8 - 16.0', '16.0 - 19.2', '19.2 - 22.4', '22.4 - 25.6', '25.6 - 28.8', '28.8 - 32.0'], 'data': [13, 6, 5, 3, 3, 4, 3, 3, 3, 4]}, 'DepartmentCategory_stringindexer_transform': {'label': ['1', '3', '2', '0'], 'data': [19, 3, 5, 20]}, 'SpecialityCategory_stringindexer_transform': {'label': ['0'], 'data': [47]}, 'PayerId_stringindexer_transform': {'label': ['0.0 - 1.9', '1.9 - 3.8', '3.8 - 5.7', '5.7 - 7.6', '7.6 - 9.5', '9.5 - 11.4', '11.4 - 13.3', '13.3 - 15.2', '15.2 - 17.1', '17.1 - 19.0'], 'data': [21, 6, 4, 4, 2, 2, 2, 2, 2, 2]}}, 'vislinegraphdata': {'ChargeDetailID': {'name': 'ChargeDetailID', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'Age': {'name': 'Age', 'series': [{'name': '1', 'value': 71}, {'name': '2', 'value': 87}, {'name': '3', 'value': 87}, {'name': '4', 'value': 47}, {'name': '5', 'value': 96}, {'name': '6', 'value': 62}, {'name': '7', 'value': 61}, {'name': '8', 'value': 67}, {'name': '9', 'value': 84}, {'name': '10', 'value': 75}, {'name': '11', 'value': 71}, {'name': '12', 'value': 88}, {'name': '13', 'value': 91}, {'name': '14', 'value': 69}, {'name': '15', 'value': 57}, {'name': '16', 'value': 78}, {'name': '17', 'value': 73}, {'name': '18', 'value': 76}, {'name': '19', 'value': 85}, {'name': '20', 'value': 65}, {'name': '21', 'value': 79}, {'name': '22', 'value': 80}, {'name': '23', 'value': 39}, {'name': '24', 'value': 34}, {'name': '25', 'value': 82}, {'name': '26', 'value': 76}, {'name': '27', 'value': 69}, {'name': '28', 'value': 77}, {'name': '29', 'value': 65}, {'name': '30', 'value': 89}, {'name': '31', 'value': 96}, {'name': '32', 'value': 84}, {'name': '33', 'value': 80}, {'name': '34', 'value': 87}, {'name': '35', 'value': 98}, {'name': '36', 'value': 64}, {'name': '37', 'value': 89}, {'name': '38', 'value': 63}, {'name': '39', 'value': 72}, {'name': '40', 'value': 76}, {'name': '41', 'value': 48}, {'name': '42', 'value': 44}, {'name': '43', 'value': 39}, {'name': '44', 'value': 60}, {'name': '45', 'value': 61}, {'name': '46', 'value': 65}, {'name': '47', 'value': 70}, {'name': '48', 'value': 0}]}, 'Gender': {'name': 'Gender', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'DateOfEntry': {'name': 'DateOfEntry', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'DateOfService': {'name': 'DateOfService', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'ProviderID': {'name': 'ProviderID', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'CPTCode': {'name': 'CPTCode', 'series': [{'name': '1', 'value': 92014}, {'name': '2', 'value': 92014}, {'name': '3', 'value': 92014}, {'name': '4', 'value': 92012}, {'name': '5', 'value': 92134}, {'name': '6', 'value': 92134}, {'name': '7', 'value': 99204}, {'name': '8', 'value': 92225}, {'name': '9', 'value': 92225}, {'name': '10', 'value': 92014}, {'name': '11', 'value': 92134}, {'name': '12', 'value': 92134}, {'name': '13', 'value': 92134}, {'name': '14', 'value': 92250}, {'name': '15', 'value': 92134}, {'name': '16', 'value': 92134}, {'name': '17', 'value': 92134}, {'name': '18', 'value': 92014}, {'name': '19', 'value': 92012}, {'name': '20', 'value': 92134}, {'name': '21', 'value': 92014}, {'name': '22', 'value': 92134}, {'name': '23', 'value': 99204}, {'name': '24', 'value': 92014}, {'name': '25', 'value': 92134}, {'name': '26', 'value': 92014}, {'name': '27', 'value': 92134}, {'name': '28', 'value': 92134}, {'name': '29', 'value': 92012}, {'name': '30', 'value': 92134}, {'name': '31', 'value': 92014}, {'name': '32', 'value': 92014}, {'name': '33', 'value': 92014}, {'name': '34', 'value': 92225}, {'name': '35', 'value': 92134}, {'name': '36', 'value': 92014}, {'name': '37', 'value': 92225}, {'name': '38', 'value': 92014}, {'name': '39', 'value': 92134}, {'name': '40', 'value': 92134}, {'name': '41', 'value': 92012}, {'name': '42', 'value': 92014}, {'name': '43', 'value': 92014}, {'name': '44', 'value': 92014}, {'name': '45', 'value': 99204}, {'name': '46', 'value': 92134}, {'name': '47', 'value': 92226}, {'name': '48', 'value': 0}]}, 'pdiagnosis1': {'name': 'pdiagnosis1', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'pdiagnosis2': {'name': 'pdiagnosis2', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'DepartmentCategory': {'name': 'DepartmentCategory', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'SpecialityCategory': {'name': 'SpecialityCategory', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'days_to_first_post': {'name': 'days_to_first_post', 'series': [{'name': '1', 'value': 9}, {'name': '2', 'value': 77}, {'name': '3', 'value': 78}, {'name': '4', 'value': 47}, {'name': '5', 'value': 42}, {'name': '6', 'value': 1}, {'name': '7', 'value': 8}, {'name': '8', 'value': 8}, {'name': '9', 'value': 32}, {'name': '10', 'value': 35}, {'name': '11', 'value': 8}, {'name': '12', 'value': 29}, {'name': '13', 'value': 33}, {'name': '14', 'value': 16}, {'name': '15', 'value': 62}, {'name': '16', 'value': 42}, {'name': '17', 'value': 40}, {'name': '18', 'value': 25}, {'name': '19', 'value': 18}, {'name': '20', 'value': 15}, {'name': '21', 'value': 36}, {'name': '22', 'value': 43}, {'name': '23', 'value': 47}, {'name': '24', 'value': 10}, {'name': '25', 'value': 36}, {'name': '26', 'value': 13}, {'name': '27', 'value': 138}, {'name': '28', 'value': 26}, {'name': '29', 'value': 14}, {'name': '30', 'value': 15}, {'name': '31', 'value': 32}, {'name': '32', 'value': 28}, {'name': '33', 'value': 26}, {'name': '34', 'value': 16}, {'name': '35', 'value': 36}, {'name': '36', 'value': 9}, {'name': '37', 'value': 21}, {'name': '38', 'value': 99}, {'name': '39', 'value': 37}, {'name': '40', 'value': 29}, {'name': '41', 'value': 13}, {'name': '42', 'value': 0}, {'name': '43', 'value': 9}, {'name': '44', 'value': 14}, {'name': '45', 'value': 8}, {'name': '46', 'value': 14}, {'name': '47', 'value': 53}, {'name': '48', 'value': 0}]}, 'PayerId': {'name': 'PayerId', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'ChargeDetailID_stringindexer_transform': {'name': 'ChargeDetailID_stringindexer_transform', 'series': [{'name': '1', 'value': 28}, {'name': '2', 'value': 45}, {'name': '3', 'value': 27}, {'name': '4', 'value': 32}, {'name': '5', 'value': 0}, {'name': '6', 'value': 40}, {'name': '7', 'value': 25}, {'name': '8', 'value': 44}, {'name': '9', 'value': 4}, {'name': '10', 'value': 29}, {'name': '11', 'value': 36}, {'name': '12', 'value': 41}, {'name': '13', 'value': 19}, {'name': '14', 'value': 21}, {'name': '15', 'value': 11}, {'name': '16', 'value': 23}, {'name': '17', 'value': 12}, {'name': '18', 'value': 24}, {'name': '19', 'value': 16}, {'name': '20', 'value': 42}, {'name': '21', 'value': 13}, {'name': '22', 'value': 3}, {'name': '23', 'value': 15}, {'name': '24', 'value': 46}, {'name': '25', 'value': 17}, {'name': '26', 'value': 2}, {'name': '27', 'value': 35}, {'name': '28', 'value': 33}, {'name': '29', 'value': 43}, {'name': '30', 'value': 26}, {'name': '31', 'value': 37}, {'name': '32', 'value': 7}, {'name': '33', 'value': 20}, {'name': '34', 'value': 22}, {'name': '35', 'value': 8}, {'name': '36', 'value': 14}, {'name': '37', 'value': 18}, {'name': '38', 'value': 38}, {'name': '39', 'value': 34}, {'name': '40', 'value': 10}, {'name': '41', 'value': 5}, {'name': '42', 'value': 30}, {'name': '43', 'value': 1}, {'name': '44', 'value': 31}, {'name': '45', 'value': 6}, {'name': '46', 'value': 39}, {'name': '47', 'value': 9}, {'name': '48', 'value': 0}]}, 'Gender_stringindexer_transform': {'name': 'Gender_stringindexer_transform', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 1}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 1}, {'name': '7', 'value': 1}, {'name': '8', 'value': 1}, {'name': '9', 'value': 0}, {'name': '10', 'value': 1}, {'name': '11', 'value': 0}, {'name': '12', 'value': 1}, {'name': '13', 'value': 1}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 1}, {'name': '17', 'value': 0}, {'name': '18', 'value': 1}, {'name': '19', 'value': 1}, {'name': '20', 'value': 1}, {'name': '21', 'value': 1}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 1}, {'name': '25', 'value': 1}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 1}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 1}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 1}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 1}, {'name': '44', 'value': 1}, {'name': '45', 'value': 1}, {'name': '46', 'value': 0}, {'name': '47', 'value': 1}, {'name': '48', 'value': 0}]}, 'DateOfEntry_dayofmonth': {'name': 'DateOfEntry_dayofmonth', 'series': [{'name': '1', 'value': 3}, {'name': '2', 'value': 5}, {'name': '3', 'value': 3}, {'name': '4', 'value': 2}, {'name': '5', 'value': 4}, {'name': '6', 'value': 5}, {'name': '7', 'value': 5}, {'name': '8', 'value': 4}, {'name': '9', 'value': 2}, {'name': '10', 'value': 5}, {'name': '11', 'value': 3}, {'name': '12', 'value': 3}, {'name': '13', 'value': 4}, {'name': '14', 'value': 4}, {'name': '15', 'value': 4}, {'name': '16', 'value': 3}, {'name': '17', 'value': 5}, {'name': '18', 'value': 4}, {'name': '19', 'value': 5}, {'name': '20', 'value': 3}, {'name': '21', 'value': 5}, {'name': '22', 'value': 3}, {'name': '23', 'value': 2}, {'name': '24', 'value': 3}, {'name': '25', 'value': 5}, {'name': '26', 'value': 5}, {'name': '27', 'value': 4}, {'name': '28', 'value': 4}, {'name': '29', 'value': 5}, {'name': '30', 'value': 3}, {'name': '31', 'value': 5}, {'name': '32', 'value': 5}, {'name': '33', 'value': 4}, {'name': '34', 'value': 4}, {'name': '35', 'value': 5}, {'name': '36', 'value': 4}, {'name': '37', 'value': 4}, {'name': '38', 'value': 5}, {'name': '39', 'value': 4}, {'name': '40', 'value': 4}, {'name': '41', 'value': 2}, {'name': '42', 'value': 5}, {'name': '43', 'value': 4}, {'name': '44', 'value': 4}, {'name': '45', 'value': 4}, {'name': '46', 'value': 5}, {'name': '47', 'value': 5}, {'name': '48', 'value': 0}]}, 'DateOfEntry_month': {'name': 'DateOfEntry_month', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 1}, {'name': '4', 'value': 1}, {'name': '5', 'value': 1}, {'name': '6', 'value': 1}, {'name': '7', 'value': 1}, {'name': '8', 'value': 1}, {'name': '9', 'value': 1}, {'name': '10', 'value': 1}, {'name': '11', 'value': 1}, {'name': '12', 'value': 1}, {'name': '13', 'value': 1}, {'name': '14', 'value': 1}, {'name': '15', 'value': 1}, {'name': '16', 'value': 1}, {'name': '17', 'value': 1}, {'name': '18', 'value': 1}, {'name': '19', 'value': 1}, {'name': '20', 'value': 1}, {'name': '21', 'value': 1}, {'name': '22', 'value': 1}, {'name': '23', 'value': 1}, {'name': '24', 'value': 1}, {'name': '25', 'value': 1}, {'name': '26', 'value': 1}, {'name': '27', 'value': 1}, {'name': '28', 'value': 1}, {'name': '29', 'value': 1}, {'name': '30', 'value': 1}, {'name': '31', 'value': 1}, {'name': '32', 'value': 1}, {'name': '33', 'value': 1}, {'name': '34', 'value': 1}, {'name': '35', 'value': 1}, {'name': '36', 'value': 1}, {'name': '37', 'value': 1}, {'name': '38', 'value': 1}, {'name': '39', 'value': 1}, {'name': '40', 'value': 1}, {'name': '41', 'value': 1}, {'name': '42', 'value': 1}, {'name': '43', 'value': 1}, {'name': '44', 'value': 1}, {'name': '45', 'value': 1}, {'name': '46', 'value': 1}, {'name': '47', 'value': 1}, {'name': '48', 'value': 0}]}, 'DateOfEntry_year': {'name': 'DateOfEntry_year', 'series': [{'name': '1', 'value': 2020}, {'name': '2', 'value': 2020}, {'name': '3', 'value': 2020}, {'name': '4', 'value': 2020}, {'name': '5', 'value': 2020}, {'name': '6', 'value': 2020}, {'name': '7', 'value': 2020}, {'name': '8', 'value': 2020}, {'name': '9', 'value': 2020}, {'name': '10', 'value': 2020}, {'name': '11', 'value': 2020}, {'name': '12', 'value': 2020}, {'name': '13', 'value': 2020}, {'name': '14', 'value': 2020}, {'name': '15', 'value': 2020}, {'name': '16', 'value': 2020}, {'name': '17', 'value': 2020}, {'name': '18', 'value': 2020}, {'name': '19', 'value': 2020}, {'name': '20', 'value': 2020}, {'name': '21', 'value': 2020}, {'name': '22', 'value': 2020}, {'name': '23', 'value': 2020}, {'name': '24', 'value': 2020}, {'name': '25', 'value': 2020}, {'name': '26', 'value': 2020}, {'name': '27', 'value': 2020}, {'name': '28', 'value': 2020}, {'name': '29', 'value': 2020}, {'name': '30', 'value': 2020}, {'name': '31', 'value': 2020}, {'name': '32', 'value': 2020}, {'name': '33', 'value': 2020}, {'name': '34', 'value': 2020}, {'name': '35', 'value': 2020}, {'name': '36', 'value': 2020}, {'name': '37', 'value': 2020}, {'name': '38', 'value': 2020}, {'name': '39', 'value': 2020}, {'name': '40', 'value': 2020}, {'name': '41', 'value': 2020}, {'name': '42', 'value': 2020}, {'name': '43', 'value': 2020}, {'name': '44', 'value': 2020}, {'name': '45', 'value': 2020}, {'name': '46', 'value': 2020}, {'name': '47', 'value': 2020}, {'name': '48', 'value': 0}]}, 'DateOfService_dayofmonth': {'name': 'DateOfService_dayofmonth', 'series': [{'name': '1', 'value': 2}, {'name': '2', 'value': 4}, {'name': '3', 'value': 2}, {'name': '4', 'value': 29}, {'name': '5', 'value': 3}, {'name': '6', 'value': 4}, {'name': '7', 'value': 4}, {'name': '8', 'value': 3}, {'name': '9', 'value': 29}, {'name': '10', 'value': 4}, {'name': '11', 'value': 2}, {'name': '12', 'value': 3}, {'name': '13', 'value': 4}, {'name': '14', 'value': 8}, {'name': '15', 'value': 3}, {'name': '16', 'value': 3}, {'name': '17', 'value': 4}, {'name': '18', 'value': 4}, {'name': '19', 'value': 4}, {'name': '20', 'value': 3}, {'name': '21', 'value': 4}, {'name': '22', 'value': 2}, {'name': '23', 'value': 29}, {'name': '24', 'value': 2}, {'name': '25', 'value': 4}, {'name': '26', 'value': 4}, {'name': '27', 'value': 4}, {'name': '28', 'value': 4}, {'name': '29', 'value': 4}, {'name': '30', 'value': 2}, {'name': '31', 'value': 4}, {'name': '32', 'value': 4}, {'name': '33', 'value': 4}, {'name': '34', 'value': 3}, {'name': '35', 'value': 4}, {'name': '36', 'value': 3}, {'name': '37', 'value': 4}, {'name': '38', 'value': 4}, {'name': '39', 'value': 3}, {'name': '40', 'value': 3}, {'name': '41', 'value': 29}, {'name': '42', 'value': 3}, {'name': '43', 'value': 3}, {'name': '44', 'value': 3}, {'name': '45', 'value': 3}, {'name': '46', 'value': 4}, {'name': '47', 'value': 4}, {'name': '48', 'value': 0}]}, 'DateOfService_month': {'name': 'DateOfService_month', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 1}, {'name': '4', 'value': 12}, {'name': '5', 'value': 1}, {'name': '6', 'value': 1}, {'name': '7', 'value': 1}, {'name': '8', 'value': 1}, {'name': '9', 'value': 12}, {'name': '10', 'value': 1}, {'name': '11', 'value': 1}, {'name': '12', 'value': 1}, {'name': '13', 'value': 1}, {'name': '14', 'value': 12}, {'name': '15', 'value': 1}, {'name': '16', 'value': 1}, {'name': '17', 'value': 1}, {'name': '18', 'value': 1}, {'name': '19', 'value': 1}, {'name': '20', 'value': 1}, {'name': '21', 'value': 1}, {'name': '22', 'value': 1}, {'name': '23', 'value': 12}, {'name': '24', 'value': 1}, {'name': '25', 'value': 1}, {'name': '26', 'value': 1}, {'name': '27', 'value': 1}, {'name': '28', 'value': 1}, {'name': '29', 'value': 1}, {'name': '30', 'value': 1}, {'name': '31', 'value': 1}, {'name': '32', 'value': 1}, {'name': '33', 'value': 1}, {'name': '34', 'value': 1}, {'name': '35', 'value': 1}, {'name': '36', 'value': 1}, {'name': '37', 'value': 1}, {'name': '38', 'value': 1}, {'name': '39', 'value': 1}, {'name': '40', 'value': 1}, {'name': '41', 'value': 12}, {'name': '42', 'value': 1}, {'name': '43', 'value': 1}, {'name': '44', 'value': 1}, {'name': '45', 'value': 1}, {'name': '46', 'value': 1}, {'name': '47', 'value': 1}, {'name': '48', 'value': 0}]}, 'DateOfService_year': {'name': 'DateOfService_year', 'series': [{'name': '1', 'value': 2020}, {'name': '2', 'value': 2020}, {'name': '3', 'value': 2020}, {'name': '4', 'value': 2019}, {'name': '5', 'value': 2020}, {'name': '6', 'value': 2020}, {'name': '7', 'value': 2020}, {'name': '8', 'value': 2020}, {'name': '9', 'value': 2019}, {'name': '10', 'value': 2020}, {'name': '11', 'value': 2020}, {'name': '12', 'value': 2020}, {'name': '13', 'value': 2020}, {'name': '14', 'value': 2019}, {'name': '15', 'value': 2020}, {'name': '16', 'value': 2020}, {'name': '17', 'value': 2020}, {'name': '18', 'value': 2020}, {'name': '19', 'value': 2020}, {'name': '20', 'value': 2020}, {'name': '21', 'value': 2020}, {'name': '22', 'value': 2020}, {'name': '23', 'value': 2019}, {'name': '24', 'value': 2020}, {'name': '25', 'value': 2020}, {'name': '26', 'value': 2020}, {'name': '27', 'value': 2020}, {'name': '28', 'value': 2020}, {'name': '29', 'value': 2020}, {'name': '30', 'value': 2020}, {'name': '31', 'value': 2020}, {'name': '32', 'value': 2020}, {'name': '33', 'value': 2020}, {'name': '34', 'value': 2020}, {'name': '35', 'value': 2020}, {'name': '36', 'value': 2020}, {'name': '37', 'value': 2020}, {'name': '38', 'value': 2020}, {'name': '39', 'value': 2020}, {'name': '40', 'value': 2020}, {'name': '41', 'value': 2019}, {'name': '42', 'value': 2020}, {'name': '43', 'value': 2020}, {'name': '44', 'value': 2020}, {'name': '45', 'value': 2020}, {'name': '46', 'value': 2020}, {'name': '47', 'value': 2020}, {'name': '48', 'value': 0}]}, 'ProviderID_stringindexer_transform': {'name': 'ProviderID_stringindexer_transform', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 0}, {'name': '4', 'value': 8}, {'name': '5', 'value': 5}, {'name': '6', 'value': 0}, {'name': '7', 'value': 9}, {'name': '8', 'value': 9}, {'name': '9', 'value': 2}, {'name': '10', 'value': 0}, {'name': '11', 'value': 2}, {'name': '12', 'value': 5}, {'name': '13', 'value': 3}, {'name': '14', 'value': 0}, {'name': '15', 'value': 2}, {'name': '16', 'value': 5}, {'name': '17', 'value': 7}, {'name': '18', 'value': 3}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 4}, {'name': '22', 'value': 1}, {'name': '23', 'value': 2}, {'name': '24', 'value': 1}, {'name': '25', 'value': 7}, {'name': '26', 'value': 7}, {'name': '27', 'value': 7}, {'name': '28', 'value': 4}, {'name': '29', 'value': 0}, {'name': '30', 'value': 2}, {'name': '31', 'value': 1}, {'name': '32', 'value': 4}, {'name': '33', 'value': 4}, {'name': '34', 'value': 3}, {'name': '35', 'value': 1}, {'name': '36', 'value': 10}, {'name': '37', 'value': 1}, {'name': '38', 'value': 3}, {'name': '39', 'value': 6}, {'name': '40', 'value': 3}, {'name': '41', 'value': 8}, {'name': '42', 'value': 6}, {'name': '43', 'value': 2}, {'name': '44', 'value': 6}, {'name': '45', 'value': 5}, {'name': '46', 'value': 0}, {'name': '47', 'value': 6}, {'name': '48', 'value': 0}]}, 'CPTCode_stringindexer_transform': {'name': 'CPTCode_stringindexer_transform', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 1}, {'name': '4', 'value': 3}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 4}, {'name': '8', 'value': 2}, {'name': '9', 'value': 2}, {'name': '10', 'value': 1}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 5}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 1}, {'name': '19', 'value': 3}, {'name': '20', 'value': 0}, {'name': '21', 'value': 1}, {'name': '22', 'value': 0}, {'name': '23', 'value': 4}, {'name': '24', 'value': 1}, {'name': '25', 'value': 0}, {'name': '26', 'value': 1}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 3}, {'name': '30', 'value': 0}, {'name': '31', 'value': 1}, {'name': '32', 'value': 1}, {'name': '33', 'value': 1}, {'name': '34', 'value': 2}, {'name': '35', 'value': 0}, {'name': '36', 'value': 1}, {'name': '37', 'value': 2}, {'name': '38', 'value': 1}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 3}, {'name': '42', 'value': 1}, {'name': '43', 'value': 1}, {'name': '44', 'value': 1}, {'name': '45', 'value': 4}, {'name': '46', 'value': 0}, {'name': '47', 'value': 6}, {'name': '48', 'value': 0}]}, 'pdiagnosis1_stringindexer_transform': {'name': 'pdiagnosis1_stringindexer_transform', 'series': [{'name': '1', 'value': 15}, {'name': '2', 'value': 5}, {'name': '3', 'value': 5}, {'name': '4', 'value': 12}, {'name': '5', 'value': 25}, {'name': '6', 'value': 2}, {'name': '7', 'value': 9}, {'name': '8', 'value': 33}, {'name': '9', 'value': 0}, {'name': '10', 'value': 10}, {'name': '11', 'value': 30}, {'name': '12', 'value': 4}, {'name': '13', 'value': 14}, {'name': '14', 'value': 22}, {'name': '15', 'value': 0}, {'name': '16', 'value': 4}, {'name': '17', 'value': 19}, {'name': '18', 'value': 32}, {'name': '19', 'value': 1}, {'name': '20', 'value': 18}, {'name': '21', 'value': 11}, {'name': '22', 'value': 1}, {'name': '23', 'value': 16}, {'name': '24', 'value': 23}, {'name': '25', 'value': 1}, {'name': '26', 'value': 17}, {'name': '27', 'value': 31}, {'name': '28', 'value': 3}, {'name': '29', 'value': 27}, {'name': '30', 'value': 28}, {'name': '31', 'value': 29}, {'name': '32', 'value': 21}, {'name': '33', 'value': 0}, {'name': '34', 'value': 7}, {'name': '35', 'value': 3}, {'name': '36', 'value': 20}, {'name': '37', 'value': 13}, {'name': '38', 'value': 2}, {'name': '39', 'value': 0}, {'name': '40', 'value': 1}, {'name': '41', 'value': 26}, {'name': '42', 'value': 8}, {'name': '43', 'value': 6}, {'name': '44', 'value': 24}, {'name': '45', 'value': 34}, {'name': '46', 'value': 0}, {'name': '47', 'value': 2}, {'name': '48', 'value': 0}]}, 'pdiagnosis2_stringindexer_transform': {'name': 'pdiagnosis2_stringindexer_transform', 'series': [{'name': '1', 'value': 20}, {'name': '2', 'value': 3}, {'name': '3', 'value': 17}, {'name': '4', 'value': 15}, {'name': '5', 'value': 27}, {'name': '6', 'value': 1}, {'name': '7', 'value': 0}, {'name': '8', 'value': 6}, {'name': '9', 'value': 32}, {'name': '10', 'value': 4}, {'name': '11', 'value': 24}, {'name': '12', 'value': 4}, {'name': '13', 'value': 26}, {'name': '14', 'value': 3}, {'name': '15', 'value': 10}, {'name': '16', 'value': 0}, {'name': '17', 'value': 8}, {'name': '18', 'value': 0}, {'name': '19', 'value': 2}, {'name': '20', 'value': 18}, {'name': '21', 'value': 0}, {'name': '22', 'value': 2}, {'name': '23', 'value': 16}, {'name': '24', 'value': 9}, {'name': '25', 'value': 2}, {'name': '26', 'value': 31}, {'name': '27', 'value': 13}, {'name': '28', 'value': 5}, {'name': '29', 'value': 7}, {'name': '30', 'value': 21}, {'name': '31', 'value': 5}, {'name': '32', 'value': 22}, {'name': '33', 'value': 19}, {'name': '34', 'value': 29}, {'name': '35', 'value': 25}, {'name': '36', 'value': 30}, {'name': '37', 'value': 23}, {'name': '38', 'value': 14}, {'name': '39', 'value': 3}, {'name': '40', 'value': 11}, {'name': '41', 'value': 1}, {'name': '42', 'value': 12}, {'name': '43', 'value': 28}, {'name': '44', 'value': 7}, {'name': '45', 'value': 8}, {'name': '46', 'value': 6}, {'name': '47', 'value': 1}, {'name': '48', 'value': 0}]}, 'DepartmentCategory_stringindexer_transform': {'name': 'DepartmentCategory_stringindexer_transform', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 1}, {'name': '6', 'value': 1}, {'name': '7', 'value': 3}, {'name': '8', 'value': 2}, {'name': '9', 'value': 2}, {'name': '10', 'value': 0}, {'name': '11', 'value': 1}, {'name': '12', 'value': 1}, {'name': '13', 'value': 1}, {'name': '14', 'value': 1}, {'name': '15', 'value': 1}, {'name': '16', 'value': 1}, {'name': '17', 'value': 1}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 1}, {'name': '21', 'value': 0}, {'name': '22', 'value': 1}, {'name': '23', 'value': 3}, {'name': '24', 'value': 0}, {'name': '25', 'value': 1}, {'name': '26', 'value': 0}, {'name': '27', 'value': 1}, {'name': '28', 'value': 1}, {'name': '29', 'value': 0}, {'name': '30', 'value': 1}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 2}, {'name': '35', 'value': 1}, {'name': '36', 'value': 0}, {'name': '37', 'value': 2}, {'name': '38', 'value': 0}, {'name': '39', 'value': 1}, {'name': '40', 'value': 1}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 3}, {'name': '46', 'value': 1}, {'name': '47', 'value': 2}, {'name': '48', 'value': 0}]}, 'SpecialityCategory_stringindexer_transform': {'name': 'SpecialityCategory_stringindexer_transform', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}]}, 'PayerId_stringindexer_transform': {'name': 'PayerId_stringindexer_transform', 'series': [{'name': '1', 'value': 2}, {'name': '2', 'value': 12}, {'name': '3', 'value': 0}, {'name': '4', 'value': 7}, {'name': '5', 'value': 0}, {'name': '6', 'value': 5}, {'name': '7', 'value': 7}, {'name': '8', 'value': 1}, {'name': '9', 'value': 19}, {'name': '10', 'value': 0}, {'name': '11', 'value': 2}, {'name': '12', 'value': 13}, {'name': '13', 'value': 0}, {'name': '14', 'value': 4}, {'name': '15', 'value': 1}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 18}, {'name': '20', 'value': 15}, {'name': '21', 'value': 0}, {'name': '22', 'value': 8}, {'name': '23', 'value': 11}, {'name': '24', 'value': 10}, {'name': '25', 'value': 0}, {'name': '26', 'value': 2}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 3}, {'name': '30', 'value': 2}, {'name': '31', 'value': 14}, {'name': '32', 'value': 17}, {'name': '33', 'value': 0}, {'name': '34', 'value': 16}, {'name': '35', 'value': 0}, {'name': '36', 'value': 1}, {'name': '37', 'value': 0}, {'name': '38', 'value': 6}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 1}, {'name': '42', 'value': 5}, {'name': '43', 'value': 4}, {'name': '44', 'value': 9}, {'name': '45', 'value': 1}, {'name': '46', 'value': 3}, {'name': '47', 'value': 6}, {'name': '48', 'value': 0}]}}}, 'ManualFE': {'FE': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '71.83', 'stddev': '15.61', 'min': '34', 'max': '98', 'missing': '0'}, 'updatedLabel': 'Age'}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateOfEntry'}, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateOfService'}, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '92546.21', 'stddev': '1758.91', 'min': '92012', 'max': '99204', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'E11.3312', 'max': 'H53.9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'B39.4', 'max': 'Z96.1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'days_to_first_post', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '30.79', 'stddev': '26.24', 'min': '0', 'max': '138', 'missing': '0'}, 'updatedLabel': 'days_to_first_post'}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '23.0', 'stddev': '13.71', 'min': '0.0', 'max': '46.0', 'missing': '0'}, 'updatedLabel': 'ChargeDetailID_stringinde...'}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}, 'updatedLabel': 'Gender_stringindexer_tran...'}, {'feature': 'DateOfEntry_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '4.0', 'stddev': '0.96', 'min': '2', 'max': '5', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_dayofmonth'}, {'feature': 'DateOfEntry_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '1.0', 'stddev': '0.0', 'min': '1', 'max': '1', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_month'}, {'feature': 'DateOfEntry_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_year'}, {'feature': 'DateOfService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '5.66', 'stddev': '7.26', 'min': '2', 'max': '29', 'missing': '0'}, 'updatedLabel': 'DateOfService_dayofmonth'}, {'feature': 'DateOfService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2.17', 'stddev': '3.43', 'min': '1', 'max': '12', 'missing': '0'}, 'updatedLabel': 'DateOfService_month'}, {'feature': 'DateOfService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2019.89', 'stddev': '0.31', 'min': '2019', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateOfService_year'}, {'feature': 'ProviderID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '3.53', 'stddev': '2.89', 'min': '0.0', 'max': '10.0', 'missing': '0'}, 'updatedLabel': 'ProviderID_stringindexer_...'}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '1.26', 'stddev': '1.5', 'min': '0.0', 'max': '6.0', 'missing': '0'}, 'updatedLabel': 'CPTCode_stringindexer_tra...'}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '13.06', 'stddev': '11.16', 'min': '0.0', 'max': '34.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis1_stringindexer...'}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '12.13', 'stddev': '10.16', 'min': '0.0', 'max': '32.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis2_stringindexer...'}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.81', 'stddev': '0.88', 'min': '0', 'max': '3', 'missing': '0'}, 'updatedLabel': 'DepartmentCategory_string...'}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}, 'updatedLabel': 'SpecialityCategory_string...'}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '4.79', 'stddev': '5.79', 'min': '0.0', 'max': '19.0', 'missing': '0'}, 'updatedLabel': 'PayerId_stringindexer_tra...'}], 'dataPercentage': '10', 'samplecount': 47, 'originalcount': 1005, 'FEGraph': [{'values': {'label': ['8406E488-7E1E-482B-80B1-4AE30E2F0CD6', '6058CDA2-9A21-4685-BA2F-8DC6AB7F8157', '5B38B5CA-A8C8-46C1-AFE3-01B32A084701', '7152B9FD-20E2-4BAA-9880-ECEEF7B7813A', '5E8A42D6-AF8A-48A6-A57C-769AA7E36D6A', '372F7455-EF3A-428E-923C-3B0465D2F474', '2071960F-CC64-4260-948F-3A0FC98E18CE', 'BE2AA60D-3002-47AB-A5AD-1FF10FA52D7A', '50503BED-86A3-4C40-9F05-A8D32F5DABE6', '7ADFBCB9-7E57-44B9-92A5-D9EA6D148794', '132E05B1-87E1-49D5-891D-7D24E6959B94', '008CD5CF-D7BE-44C6-AB9A-1EA061BE4800', 'A784EB62-A0F5-463D-BD9F-412AF036DDD7', 'C481E307-0A94-467B-BF82-53D34728427E', '43846138-A55C-4877-863F-C1AC659A3F44', '80CB21C5-3FFF-4F35-B228-4A1ABB4D2015', '6BDD5CED-1DE1-4774-BC24-03127487C3D3', '4D15A79D-1B97-461B-8994-C9DB9BBFE5FF', 'DB8DA258-E24D-43A0-BE20-9DABC58150D6', '327659EE-D609-4ED5-83FB-A87DF7EAC224', '1E6165E2-7C9B-4680-972C-72BFD783CCA6', 'A21E71B8-5267-4260-B174-EBEACF6765A6', '04027CE6-725D-45D3-B3F5-FE1684208F6F', 'BB460B17-E6E0-4B21-862A-DDBD7B224812', '234584C4-2DB1-494F-9434-563A492CD2E9', '4DC676C3-A9E6-428A-BBA0-00D3B4BA241F', '97022FC6-EEF2-46E2-8EDF-FD14DA109466', 'AA4F394E-5E18-450E-ABEC-E45E7C350417', 'D68F312D-BC27-43BC-A9ED-D9ED783E8B36', 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', '3D69FE56-3157-4826-AB50-74B461494603', '103BDFEF-C800-4C4C-9E16-AF05608834B4', 'ECC0BDBA-B14F-4DAE-AA87-AEA5CE026D26', '51BEF171-E2C4-49D5-A293-03625E141B41', '84CCF9C2-928B-42E9-B28C-56059E818044', 'B7EEB9D2-7A57-41BB-A4A3-C59F730DF615', '4E99A4C7-0358-4B6C-BBBB-C87D18F4BB12', 'CE28AB61-0492-4A59-9574-AA9AF276B541', '2273E2B7-C4C8-487C-BCB2-30B7C5E1B89F', '8E183988-EB23-4169-93CB-CBBC35355A5F', '9EF8EAF8-5973-4D12-B1FC-7D710FA6D6BC', '4514D14C-EE3B-4D1A-94DC-F729CBFEFA40', '4939A5A3-0D69-4371-886A-C26434D76ED8', '2E51A705-F0D3-4772-945D-9A18D5EA4310', '6465F86A-70DB-4E2D-ABA8-8DBCD72CAFBB', '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', '58FEB416-06D5-49AA-A463-5A7D48BE2621'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'ChargeDetailID', 'count': 47, 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2'}, {'values': {'label': ['34.0 - 40.4', '40.4 - 46.8', '46.8 - 53.2', '53.2 - 59.6', '59.6 - 66.0', '66.0 - 72.4', '72.4 - 78.8', '78.8 - 85.2', '85.2 - 91.6', '91.6 - 98.0'], 'data': [3, 1, 2, 1, 9, 7, 7, 7, 7, 3]}, 'label': 'Age', 'count': 10, 'min': 34, 'max': 98}, {'values': {'label': ['F', 'M'], 'data': [26, 21]}, 'label': 'Gender', 'count': 2, 'min': 'F', 'max': 'M'}, {'values': {'label': ['2020-01-05 00:00:00', '2020-01-04 00:00:00', '2020-01-03 00:00:00', '2020-01-02 00:00:00'], 'data': [17, 17, 9, 4]}, 'label': 'DateOfEntry', 'count': 4, 'min': '', 'max': ''}, {'values': {'label': ['2020-01-04 00:00:00', '2020-01-03 00:00:00', '2019-12-08 00:00:00', '2019-12-29 00:00:00', '2020-01-02 00:00:00'], 'data': [22, 14, 1, 4, 6]}, 'label': 'DateOfService', 'count': 5, 'min': '', 'max': ''}, {'values': {'label': ['7323A5D3-9249-4F98-9E04-2A53151C8290', '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'A724E957-9B9A-435F-A593-D3272B479E7B', '7A9BF809-317C-435B-9CF4-868B78F71063', '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', '0AC82717-3661-4548-8119-193431DEDEDA', '69294C2D-F7DC-473E-AA9E-53A44DC9B067', '03151B5D-685B-4211-B01F-7BBB1C4A8A49', '530C8C66-2D52-4067-8526-D4F3C94372F0'], 'data': [2, 8, 5, 4, 2, 4, 1, 7, 4, 6, 4]}, 'label': 'ProviderID', 'count': 11, 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3'}, {'values': {'label': ['92014', '92134', '92250', '99204', '92012', '92226', '92225'], 'data': [16, 18, 1, 3, 4, 1, 4]}, 'label': 'CPTCode', 'count': 7, 'min': '92012', 'max': '99204'}, {'values': {'label': ['H33.313', 'H35.3132', 'H43.821', 'E11.3513', 'H35.3122', 'H34.8110', 'H35.3221', 'H34.8310', 'H40.1231', 'H25.89', 'H35.373', 'G45.9', 'E11.3312', 'H35.3211', 'E11.3522', 'E11.9', 'H35.3212', 'H35.3112', 'H35.3210', 'H04.123', 'H33.002', 'H35.352', 'H35.413', 'E11.3413', 'H35.033', 'H35.371', 'H53.9', 'H35.3213', 'E11.3313', 'H33.302', 'H43.813', 'H35.3231', 'H40.1130', 'H33.321', 'H35.712'], 'data': [1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1]}, 'label': 'pdiagnosis1', 'count': 35, 'min': 'E11.3312', 'max': 'H53.9'}, {'values': {'label': ['D31.31', 'H33.313', 'H35.372', 'H35.3132', 'H35.3134', 'H35.3122', 'H35.3221', 'B39.4', 'H35.342', 'H35.373', 'H43.11', 'H35.3211', 'H34.8322', 'H44.23', 'E11.9', 'H35.461', 'H35.052', 'H33.002', 'H25.12', 'H35.352', 'H35.413', 'H35.063', 'E11.3291', 'H35.63', 'H35.033', 'H35.371', 'H35.70', 'H43.12', 'H35.463', 'H43.812', 'E11.3313', 'H43.813', 'Z96.1'], 'data': [2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 3, 1, 1, 1, 2, 1, 1, 1]}, 'label': 'pdiagnosis2', 'count': 33, 'min': 'B39.4', 'max': 'Z96.1'}, {'values': {'label': ['Pulmonary ', 'Orthopedic', 'Cardiology', 'Emergency'], 'data': [20, 3, 5, 19]}, 'label': 'DepartmentCategory', 'count': 4, 'min': 'Cardiology', 'max': 'Pulmonary '}, {'values': {'label': ['Ophtamology'], 'data': [47]}, 'label': 'SpecialityCategory', 'count': 1, 'min': 'Ophtamology', 'max': 'Ophtamology'}, {'values': {'label': ['0.0 - 13.8', '13.8 - 27.6', '27.6 - 41.4', '41.4 - 55.2', '55.2 - 69.0', '69.0 - 82.8', '82.8 - 96.6', '96.6 - 110.4', '110.4 - 124.2', '124.2 - 138.0'], 'data': [12, 12, 12, 6, 1, 2, 0, 1, 0, 1]}, 'label': 'days_to_first_post', 'count': 10, 'min': 0, 'max': 138}, {'values': {'label': ['75F80B5C-AD0E-4A1C-946C-07AEB69B7F9B', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', '2AF763CD-E2CF-4E69-80E5-B00EC12B032F', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'AD96772F-54C7-4750-86B1-1BD565B55244', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '91A3F98C-94F0-4427-8B42-106B6D6C707E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '315C773B-4EF1-4164-AB43-6817D993329F', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'B9F6F179-DD6F-486B-98EE-C207EE60842E', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4'], 'data': [1, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 5, 1, 2, 1, 16, 2, 2, 1, 2]}, 'label': 'PayerId', 'count': 20, 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52'}, {'values': {'label': ['0.0 - 4.6', '4.6 - 9.2', '9.2 - 13.8', '13.8 - 18.4', '18.4 - 23.0', '23.0 - 27.6', '27.6 - 32.2', '32.2 - 36.8', '36.8 - 41.4', '41.4 - 46.0'], 'data': [5, 5, 4, 5, 4, 5, 5, 4, 5, 5]}, 'label': 'ChargeDetailID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 46}, {'values': {'label': ['1', '0'], 'data': [21, 26]}, 'label': 'Gender_stringindexer_transform', 'count': 2, 'min': '0', 'max': '1'}, {'values': {'label': ['3', '5', '4', '2'], 'data': [9, 17, 17, 4]}, 'label': 'DateOfEntry_dayofmonth', 'count': 4, 'min': '2', 'max': '5'}, {'values': {'label': ['1'], 'data': [47]}, 'label': 'DateOfEntry_month', 'count': 1, 'min': '1', 'max': '1'}, {'values': {'label': ['2020'], 'data': [47]}, 'label': 'DateOfEntry_year', 'count': 1, 'min': '2020', 'max': '2020'}, {'values': {'label': ['3', '4', '8', '29', '2'], 'data': [14, 22, 1, 4, 6]}, 'label': 'DateOfService_dayofmonth', 'count': 5, 'min': '2', 'max': '29'}, {'values': {'label': ['12', '1'], 'data': [5, 42]}, 'label': 'DateOfService_month', 'count': 2, 'min': '1', 'max': '12'}, {'values': {'label': ['2019', '2020'], 'data': [5, 42]}, 'label': 'DateOfService_year', 'count': 2, 'min': '2019', 'max': '2020'}, {'values': {'label': ['0.0 - 1.0', '1.0 - 2.0', '2.0 - 3.0', '3.0 - 4.0', '4.0 - 5.0', '5.0 - 6.0', '6.0 - 7.0', '7.0 - 8.0', '8.0 - 9.0', '9.0 - 10.0'], 'data': [8, 7, 6, 5, 4, 4, 4, 4, 2, 3]}, 'label': 'ProviderID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 10}, {'values': {'label': ['0.0', '1.0', '4.0', '3.0', '2.0', '6.0', '5.0'], 'data': [18, 16, 3, 4, 4, 1, 1]}, 'label': 'CPTCode_stringindexer_transform', 'count': 7, 'min': '0.0', 'max': '6.0'}, {'values': {'label': ['0.0 - 3.4', '3.4 - 6.8', '6.8 - 10.2', '10.2 - 13.6', '13.6 - 17.0', '17.0 - 20.4', '20.4 - 23.8', '23.8 - 27.2', '27.2 - 30.6', '30.6 - 34.0'], 'data': [14, 5, 4, 3, 3, 4, 3, 4, 3, 4]}, 'label': 'pdiagnosis1_stringindexer_transform', 'count': 10, 'min': 0, 'max': 34}, {'values': {'label': ['0.0 - 3.2', '3.2 - 6.4', '6.4 - 9.6', '9.6 - 12.8', '12.8 - 16.0', '16.0 - 19.2', '19.2 - 22.4', '22.4 - 25.6', '25.6 - 28.8', '28.8 - 32.0'], 'data': [13, 6, 5, 3, 3, 4, 3, 3, 3, 4]}, 'label': 'pdiagnosis2_stringindexer_transform', 'count': 10, 'min': 0, 'max': 32}, {'values': {'label': ['1', '3', '2', '0'], 'data': [19, 3, 5, 20]}, 'label': 'DepartmentCategory_stringindexer_transform', 'count': 4, 'min': '0', 'max': '3'}, {'values': {'label': ['0'], 'data': [47]}, 'label': 'SpecialityCategory_stringindexer_transform', 'count': 1, 'min': '0', 'max': '0'}, {'values': {'label': ['0.0 - 1.9', '1.9 - 3.8', '3.8 - 5.7', '5.7 - 7.6', '7.6 - 9.5', '9.5 - 11.4', '11.4 - 13.3', '13.3 - 15.2', '15.2 - 17.1', '17.1 - 19.0'], 'data': [21, 6, 4, 4, 2, 2, 2, 2, 2, 2]}, 'label': 'PayerId_stringindexer_transform', 'count': 10, 'min': 0, 'max': 19}], 'FESamplePath': '/FileStore/platform/sampleData/60824e962e36424d53db3377_ManualFE/part-00000-tid-8790912989024766106-9063b124-4e5f-4e0c-aa79-e07bb5fdcc72-482370-1-c000.csv', 'manualfesamplepath': '/FileStore/platform/sampleData/60824e962e36424d53db3377_ManualFE/part-00000-tid-8790912989024766106-9063b124-4e5f-4e0c-aa79-e07bb5fdcc72-482370-1-c000.csv', 'stageId': '60824e962e36424d53db3377', 'stage_id': '60824e962e36424d53db3377', 'status': 1, 'autoFE_flag': True, 'userid': '5ec8228938f3b7a0e925d07e', 'message': 'Successfully executed', 'process_type': 'Auto FE', 'sampledata': [{'ChargeDetailID': '8406E488-7E1E-482B-80B1-4AE30E2F0CD6', 'Age': 71, 'Gender': 'F', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1577923200000.0, 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92014', 'pdiagnosis1': 'H43.821', 'pdiagnosis2': 'E11.3313', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 9, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'ChargeDetailID_stringindexer_transform': 28, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 2, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 1, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 15, 'pdiagnosis2_stringindexer_transform': 20, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '6058CDA2-9A21-4685-BA2F-8DC6AB7F8157', 'Age': 87, 'Gender': 'M', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92014', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.371', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 77, 'PayerId': '91A3F98C-94F0-4427-8B42-106B6D6C707E', 'ChargeDetailID_stringindexer_transform': 45, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 1, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 3, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 12}, {'ChargeDetailID': '5B38B5CA-A8C8-46C1-AFE3-01B32A084701', 'Age': 87, 'Gender': 'F', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1577923200000.0, 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92014', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'Z96.1', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 78, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 27, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 2, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 0, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 17, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '7152B9FD-20E2-4BAA-9880-ECEEF7B7813A', 'Age': 47, 'Gender': 'F', 'DateOfEntry': 1577923200000.0, 'DateOfService': 1577577600000.0, 'ProviderID': '7323A5D3-9249-4F98-9E04-2A53151C8290', 'CPTCode': '92012', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H35.63', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 47, 'PayerId': '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'ChargeDetailID_stringindexer_transform': 32, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 2, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 29, 'DateOfService_month': 12, 'DateOfService_year': 2019, 'ProviderID_stringindexer_transform': 8, 'CPTCode_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 15, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': '5E8A42D6-AF8A-48A6-A57C-769AA7E36D6A', 'Age': 96, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '7A9BF809-317C-435B-9CF4-868B78F71063', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.813', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 42, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 5, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 25, 'pdiagnosis2_stringindexer_transform': 27, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '372F7455-EF3A-428E-923C-3B0465D2F474', 'Age': 62, 'Gender': 'M', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92134', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 1, 'PayerId': 'B9F6F179-DD6F-486B-98EE-C207EE60842E', 'ChargeDetailID_stringindexer_transform': 40, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 0, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': '2071960F-CC64-4260-948F-3A0FC98E18CE', 'Age': 61, 'Gender': 'M', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', 'CPTCode': '99204', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H35.033', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 8, 'PayerId': '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'ChargeDetailID_stringindexer_transform': 25, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 9, 'CPTCode_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': 'BE2AA60D-3002-47AB-A5AD-1FF10FA52D7A', 'Age': 67, 'Gender': 'M', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', 'CPTCode': '92225', 'pdiagnosis1': 'H33.313', 'pdiagnosis2': 'H35.373', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 8, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': 44, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 9, 'CPTCode_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 33, 'pdiagnosis2_stringindexer_transform': 6, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '50503BED-86A3-4C40-9F05-A8D32F5DABE6', 'Age': 84, 'Gender': 'F', 'DateOfEntry': 1577923200000.0, 'DateOfService': 1577577600000.0, 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92225', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H33.313', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 32, 'PayerId': '2AF763CD-E2CF-4E69-80E5-B00EC12B032F', 'ChargeDetailID_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 2, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 29, 'DateOfService_month': 12, 'DateOfService_year': 2019, 'ProviderID_stringindexer_transform': 2, 'CPTCode_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 32, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 19}, {'ChargeDetailID': '7ADFBCB9-7E57-44B9-92A5-D9EA6D148794', 'Age': 75, 'Gender': 'M', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92014', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'E11.3291', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 35, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 29, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 0, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 4, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '132E05B1-87E1-49D5-891D-7D24E6959B94', 'Age': 71, 'Gender': 'F', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1577923200000.0, 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3221', 'pdiagnosis2': 'H44.23', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 8, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'ChargeDetailID_stringindexer_transform': 36, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 2, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 2, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 30, 'pdiagnosis2_stringindexer_transform': 24, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '008CD5CF-D7BE-44C6-AB9A-1EA061BE4800', 'Age': 88, 'Gender': 'M', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '7A9BF809-317C-435B-9CF4-868B78F71063', 'CPTCode': '92134', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'E11.3291', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 29, 'PayerId': 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'ChargeDetailID_stringindexer_transform': 41, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 5, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 4, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 13}, {'ChargeDetailID': 'A784EB62-A0F5-463D-BD9F-412AF036DDD7', 'Age': 91, 'Gender': 'M', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578096000000.0, 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3213', 'pdiagnosis2': 'H35.352', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 33, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 19, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 3, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 26, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C481E307-0A94-467B-BF82-53D34728427E', 'Age': 69, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1575763200000.0, 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92250', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.371', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 16, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'ChargeDetailID_stringindexer_transform': 21, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 8, 'DateOfService_month': 12, 'DateOfService_year': 2019, 'ProviderID_stringindexer_transform': 0, 'CPTCode_stringindexer_transform': 5, 'pdiagnosis1_stringindexer_transform': 22, 'pdiagnosis2_stringindexer_transform': 3, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '43846138-A55C-4877-863F-C1AC659A3F44', 'Age': 57, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92134', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.063', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 62, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': 11, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 2, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 10, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '80CB21C5-3FFF-4F35-B228-4A1ABB4D2015', 'Age': 78, 'Gender': 'M', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '7A9BF809-317C-435B-9CF4-868B78F71063', 'CPTCode': '92134', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.033', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 42, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 23, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 5, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '6BDD5CED-1DE1-4774-BC24-03127487C3D3', 'Age': 73, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '69294C2D-F7DC-473E-AA9E-53A44DC9B067', 'CPTCode': '92134', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H43.812', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 40, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 12, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 7, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 19, 'pdiagnosis2_stringindexer_transform': 8, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '4D15A79D-1B97-461B-8994-C9DB9BBFE5FF', 'Age': 76, 'Gender': 'M', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578096000000.0, 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92014', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.033', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 25, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 24, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 3, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 32, 'pdiagnosis2_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'DB8DA258-E24D-43A0-BE20-9DABC58150D6', 'Age': 85, 'Gender': 'M', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92012', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 18, 'PayerId': '193BA5E7-F797-4FD1-A9AB-27590E630FCB', 'ChargeDetailID_stringindexer_transform': 16, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 0, 'CPTCode_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 2, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 18}, {'ChargeDetailID': '327659EE-D609-4ED5-83FB-A87DF7EAC224', 'Age': 65, 'Gender': 'M', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92134', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H35.413', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 15, 'PayerId': '315C773B-4EF1-4164-AB43-6817D993329F', 'ChargeDetailID_stringindexer_transform': 42, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 0, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 18, 'pdiagnosis2_stringindexer_transform': 18, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 15}, {'ChargeDetailID': '1E6165E2-7C9B-4680-972C-72BFD783CCA6', 'Age': 79, 'Gender': 'M', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CPTCode': '92014', 'pdiagnosis1': 'E11.9', 'pdiagnosis2': 'H35.033', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 36, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 13, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 4, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A21E71B8-5267-4260-B174-EBEACF6765A6', 'Age': 80, 'Gender': 'F', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1577923200000.0, 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 43, 'PayerId': 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'ChargeDetailID_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 2, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 1, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 2, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 8}, {'ChargeDetailID': '04027CE6-725D-45D3-B3F5-FE1684208F6F', 'Age': 39, 'Gender': 'F', 'DateOfEntry': 1577923200000.0, 'DateOfService': 1577577600000.0, 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '99204', 'pdiagnosis1': 'H35.413', 'pdiagnosis2': 'H35.463', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 47, 'PayerId': '75F80B5C-AD0E-4A1C-946C-07AEB69B7F9B', 'ChargeDetailID_stringindexer_transform': 15, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 2, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 29, 'DateOfService_month': 12, 'DateOfService_year': 2019, 'ProviderID_stringindexer_transform': 2, 'CPTCode_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 16, 'pdiagnosis2_stringindexer_transform': 16, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 11}, {'ChargeDetailID': 'BB460B17-E6E0-4B21-862A-DDBD7B224812', 'Age': 34, 'Gender': 'M', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1577923200000.0, 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92014', 'pdiagnosis1': 'H25.89', 'pdiagnosis2': 'H33.002', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 10, 'PayerId': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'ChargeDetailID_stringindexer_transform': 46, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 2, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 1, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 23, 'pdiagnosis2_stringindexer_transform': 9, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 10}, {'ChargeDetailID': '234584C4-2DB1-494F-9434-563A492CD2E9', 'Age': 82, 'Gender': 'M', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '69294C2D-F7DC-473E-AA9E-53A44DC9B067', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 36, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 17, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 7, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 2, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '4DC676C3-A9E6-428A-BBA0-00D3B4BA241F', 'Age': 76, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '69294C2D-F7DC-473E-AA9E-53A44DC9B067', 'CPTCode': '92014', 'pdiagnosis1': 'H04.123', 'pdiagnosis2': 'H35.3211', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 13, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'ChargeDetailID_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 7, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 17, 'pdiagnosis2_stringindexer_transform': 31, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '97022FC6-EEF2-46E2-8EDF-FD14DA109466', 'Age': 69, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '69294C2D-F7DC-473E-AA9E-53A44DC9B067', 'CPTCode': '92134', 'pdiagnosis1': 'H35.352', 'pdiagnosis2': 'H35.372', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 138, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 35, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 7, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 31, 'pdiagnosis2_stringindexer_transform': 13, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'AA4F394E-5E18-450E-ABEC-E45E7C350417', 'Age': 77, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'E11.9', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 26, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 33, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 4, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 5, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D68F312D-BC27-43BC-A9ED-D9ED783E8B36', 'Age': 65, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92012', 'pdiagnosis1': 'G45.9', 'pdiagnosis2': 'D31.31', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 14, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'ChargeDetailID_stringindexer_transform': 43, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 0, 'CPTCode_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 27, 'pdiagnosis2_stringindexer_transform': 7, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', 'Age': 89, 'Gender': 'M', 'DateOfEntry': 1578009600000.0, 'DateOfService': 1577923200000.0, 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92134', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H34.8322', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 15, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'ChargeDetailID_stringindexer_transform': 26, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 3, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 2, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 2, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 28, 'pdiagnosis2_stringindexer_transform': 21, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '3D69FE56-3157-4826-AB50-74B461494603', 'Age': 96, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92014', 'pdiagnosis1': 'H35.3210', 'pdiagnosis2': 'E11.9', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 32, 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'ChargeDetailID_stringindexer_transform': 37, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 1, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 29, 'pdiagnosis2_stringindexer_transform': 5, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 14}, {'ChargeDetailID': '103BDFEF-C800-4C4C-9E16-AF05608834B4', 'Age': 84, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CPTCode': '92014', 'pdiagnosis1': 'H40.1130', 'pdiagnosis2': 'H35.3132', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 28, 'PayerId': 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'ChargeDetailID_stringindexer_transform': 7, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 4, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 21, 'pdiagnosis2_stringindexer_transform': 22, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 17}, {'ChargeDetailID': 'ECC0BDBA-B14F-4DAE-AA87-AEA5CE026D26', 'Age': 80, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CPTCode': '92014', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H25.12', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 26, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 20, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 4, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 19, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '51BEF171-E2C4-49D5-A293-03625E141B41', 'Age': 87, 'Gender': 'M', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92225', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.342', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 16, 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'ChargeDetailID_stringindexer_transform': 22, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 3, 'CPTCode_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 29, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 16}, {'ChargeDetailID': '84CCF9C2-928B-42E9-B28C-56059E818044', 'Age': 98, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 36, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 8, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 1, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 25, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'B7EEB9D2-7A57-41BB-A4A3-C59F730DF615', 'Age': 64, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'CPTCode': '92014', 'pdiagnosis1': 'H33.321', 'pdiagnosis2': 'H35.461', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 9, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': 14, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 10, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 20, 'pdiagnosis2_stringindexer_transform': 30, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '4E99A4C7-0358-4B6C-BBBB-C87D18F4BB12', 'Age': 89, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '0AC82717-3661-4548-8119-193431DEDEDA', 'CPTCode': '92225', 'pdiagnosis1': 'H53.9', 'pdiagnosis2': 'B39.4', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 21, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 18, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 1, 'CPTCode_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 23, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'CE28AB61-0492-4A59-9574-AA9AF276B541', 'Age': 63, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92014', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.12', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 99, 'PayerId': '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'ChargeDetailID_stringindexer_transform': 38, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 3, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 14, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '2273E2B7-C4C8-487C-BCB2-30B7C5E1B89F', 'Age': 72, 'Gender': 'F', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '530C8C66-2D52-4067-8526-D4F3C94372F0', 'CPTCode': '92134', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.371', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 37, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 34, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 6, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 3, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '8E183988-EB23-4169-93CB-CBBC35355A5F', 'Age': 76, 'Gender': 'M', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': 'A724E957-9B9A-435F-A593-D3272B479E7B', 'CPTCode': '92134', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3134', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 29, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'ChargeDetailID_stringindexer_transform': 10, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 3, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 11, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9EF8EAF8-5973-4D12-B1FC-7D710FA6D6BC', 'Age': 48, 'Gender': 'F', 'DateOfEntry': 1577923200000.0, 'DateOfService': 1577577600000.0, 'ProviderID': '7323A5D3-9249-4F98-9E04-2A53151C8290', 'CPTCode': '92012', 'pdiagnosis1': 'E11.3522', 'pdiagnosis2': 'H43.11', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 13, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 2, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 29, 'DateOfService_month': 12, 'DateOfService_year': 2019, 'ProviderID_stringindexer_transform': 8, 'CPTCode_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 26, 'pdiagnosis2_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '4514D14C-EE3B-4D1A-94DC-F729CBFEFA40', 'Age': 44, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '530C8C66-2D52-4067-8526-D4F3C94372F0', 'CPTCode': '92014', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.052', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 0, 'PayerId': 'B9F6F179-DD6F-486B-98EE-C207EE60842E', 'ChargeDetailID_stringindexer_transform': 30, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 6, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 12, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': '4939A5A3-0D69-4371-886A-C26434D76ED8', 'Age': 39, 'Gender': 'M', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'CPTCode': '92014', 'pdiagnosis1': 'H35.712', 'pdiagnosis2': 'H35.70', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 9, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'ChargeDetailID_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 2, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 28, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '2E51A705-F0D3-4772-945D-9A18D5EA4310', 'Age': 60, 'Gender': 'M', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '530C8C66-2D52-4067-8526-D4F3C94372F0', 'CPTCode': '92014', 'pdiagnosis1': 'H40.1231', 'pdiagnosis2': 'D31.31', 'DepartmentCategory': 'Pulmonary ', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 14, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'ChargeDetailID_stringindexer_transform': 31, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 6, 'CPTCode_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 24, 'pdiagnosis2_stringindexer_transform': 7, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 9}, {'ChargeDetailID': '6465F86A-70DB-4E2D-ABA8-8DBCD72CAFBB', 'Age': 61, 'Gender': 'M', 'DateOfEntry': 1578096000000.0, 'DateOfService': 1578009600000.0, 'ProviderID': '7A9BF809-317C-435B-9CF4-868B78F71063', 'CPTCode': '99204', 'pdiagnosis1': 'H33.302', 'pdiagnosis2': 'H43.812', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 8, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'ChargeDetailID_stringindexer_transform': 6, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 4, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 3, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 5, 'CPTCode_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 34, 'pdiagnosis2_stringindexer_transform': 8, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'Age': 65, 'Gender': 'F', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'CPTCode': '92134', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.373', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 14, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'ChargeDetailID_stringindexer_transform': 39, 'Gender_stringindexer_transform': 0, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 0, 'CPTCode_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 6, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '58FEB416-06D5-49AA-A463-5A7D48BE2621', 'Age': 70, 'Gender': 'M', 'DateOfEntry': 1578182400000.0, 'DateOfService': 1578096000000.0, 'ProviderID': '530C8C66-2D52-4067-8526-D4F3C94372F0', 'CPTCode': '92226', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'days_to_first_post': 53, 'PayerId': '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'ChargeDetailID_stringindexer_transform': 9, 'Gender_stringindexer_transform': 1, 'DateOfEntry_dayofmonth': 5, 'DateOfEntry_month': 1, 'DateOfEntry_year': 2020, 'DateOfService_dayofmonth': 4, 'DateOfService_month': 1, 'DateOfService_year': 2020, 'ProviderID_stringindexer_transform': 6, 'CPTCode_stringindexer_transform': 6, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}], 'attributesFT': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '71.83', 'stddev': '15.61', 'min': '34', 'max': '98', 'missing': '0'}}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False}, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False}, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '92546.21', 'stddev': '1758.91', 'min': '92012', 'max': '99204', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'E11.3312', 'max': 'H53.9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'B39.4', 'max': 'Z96.1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'days_to_first_post', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '30.79', 'stddev': '26.24', 'min': '0', 'max': '138', 'missing': '0'}}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '23.0', 'stddev': '13.71', 'min': '0.0', 'max': '46.0', 'missing': '0'}}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}}, {'feature': 'DateOfEntry_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '4.0', 'stddev': '0.96', 'min': '2', 'max': '5', 'missing': '0'}}, {'feature': 'DateOfEntry_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '1.0', 'stddev': '0.0', 'min': '1', 'max': '1', 'missing': '0'}}, {'feature': 'DateOfEntry_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}}, {'feature': 'DateOfService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '5.66', 'stddev': '7.26', 'min': '2', 'max': '29', 'missing': '0'}}, {'feature': 'DateOfService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2.17', 'stddev': '3.43', 'min': '1', 'max': '12', 'missing': '0'}}, {'feature': 'DateOfService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2019.89', 'stddev': '0.31', 'min': '2019', 'max': '2020', 'missing': '0'}}, {'feature': 'ProviderID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '3.53', 'stddev': '2.89', 'min': '0.0', 'max': '10.0', 'missing': '0'}}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '1.26', 'stddev': '1.5', 'min': '0.0', 'max': '6.0', 'missing': '0'}}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '13.06', 'stddev': '11.16', 'min': '0.0', 'max': '34.0', 'missing': '0'}}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '12.13', 'stddev': '10.16', 'min': '0.0', 'max': '32.0', 'missing': '0'}}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.81', 'stddev': '0.88', 'min': '0', 'max': '3', 'missing': '0'}}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '4.79', 'stddev': '5.79', 'min': '0.0', 'max': '19.0', 'missing': '0'}}], 'path_AutoSample': '/FileStore/platform/sampleData/60824e962e36424d53db3377_AutoFE/part-00000-tid-2315973788566987767-9bdba334-e9f6-4cf5-8935-de1b36ce2a7b-482168-1-c000.csv', 'listOfGraph': [{'values': {'label': ['8406E488-7E1E-482B-80B1-4AE30E2F0CD6', '6058CDA2-9A21-4685-BA2F-8DC6AB7F8157', '5B38B5CA-A8C8-46C1-AFE3-01B32A084701', '7152B9FD-20E2-4BAA-9880-ECEEF7B7813A', '5E8A42D6-AF8A-48A6-A57C-769AA7E36D6A', '372F7455-EF3A-428E-923C-3B0465D2F474', '2071960F-CC64-4260-948F-3A0FC98E18CE', 'BE2AA60D-3002-47AB-A5AD-1FF10FA52D7A', '50503BED-86A3-4C40-9F05-A8D32F5DABE6', '7ADFBCB9-7E57-44B9-92A5-D9EA6D148794', '132E05B1-87E1-49D5-891D-7D24E6959B94', '008CD5CF-D7BE-44C6-AB9A-1EA061BE4800', 'A784EB62-A0F5-463D-BD9F-412AF036DDD7', 'C481E307-0A94-467B-BF82-53D34728427E', '43846138-A55C-4877-863F-C1AC659A3F44', '80CB21C5-3FFF-4F35-B228-4A1ABB4D2015', '6BDD5CED-1DE1-4774-BC24-03127487C3D3', '4D15A79D-1B97-461B-8994-C9DB9BBFE5FF', 'DB8DA258-E24D-43A0-BE20-9DABC58150D6', '327659EE-D609-4ED5-83FB-A87DF7EAC224', '1E6165E2-7C9B-4680-972C-72BFD783CCA6', 'A21E71B8-5267-4260-B174-EBEACF6765A6', '04027CE6-725D-45D3-B3F5-FE1684208F6F', 'BB460B17-E6E0-4B21-862A-DDBD7B224812', '234584C4-2DB1-494F-9434-563A492CD2E9', '4DC676C3-A9E6-428A-BBA0-00D3B4BA241F', '97022FC6-EEF2-46E2-8EDF-FD14DA109466', 'AA4F394E-5E18-450E-ABEC-E45E7C350417', 'D68F312D-BC27-43BC-A9ED-D9ED783E8B36', 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', '3D69FE56-3157-4826-AB50-74B461494603', '103BDFEF-C800-4C4C-9E16-AF05608834B4', 'ECC0BDBA-B14F-4DAE-AA87-AEA5CE026D26', '51BEF171-E2C4-49D5-A293-03625E141B41', '84CCF9C2-928B-42E9-B28C-56059E818044', 'B7EEB9D2-7A57-41BB-A4A3-C59F730DF615', '4E99A4C7-0358-4B6C-BBBB-C87D18F4BB12', 'CE28AB61-0492-4A59-9574-AA9AF276B541', '2273E2B7-C4C8-487C-BCB2-30B7C5E1B89F', '8E183988-EB23-4169-93CB-CBBC35355A5F', '9EF8EAF8-5973-4D12-B1FC-7D710FA6D6BC', '4514D14C-EE3B-4D1A-94DC-F729CBFEFA40', '4939A5A3-0D69-4371-886A-C26434D76ED8', '2E51A705-F0D3-4772-945D-9A18D5EA4310', '6465F86A-70DB-4E2D-ABA8-8DBCD72CAFBB', '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', '58FEB416-06D5-49AA-A463-5A7D48BE2621'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'ChargeDetailID', 'count': 47, 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2'}, {'values': {'label': ['34.0 - 40.4', '40.4 - 46.8', '46.8 - 53.2', '53.2 - 59.6', '59.6 - 66.0', '66.0 - 72.4', '72.4 - 78.8', '78.8 - 85.2', '85.2 - 91.6', '91.6 - 98.0'], 'data': [3, 1, 2, 1, 9, 7, 7, 7, 7, 3]}, 'label': 'Age', 'count': 10, 'min': 34, 'max': 98}, {'values': {'label': ['F', 'M'], 'data': [26, 21]}, 'label': 'Gender', 'count': 2, 'min': 'F', 'max': 'M'}, {'values': {'label': ['2020-01-05 00:00:00', '2020-01-04 00:00:00', '2020-01-03 00:00:00', '2020-01-02 00:00:00'], 'data': [17, 17, 9, 4]}, 'label': 'DateOfEntry', 'count': 4, 'min': '', 'max': ''}, {'values': {'label': ['2020-01-04 00:00:00', '2020-01-03 00:00:00', '2019-12-08 00:00:00', '2019-12-29 00:00:00', '2020-01-02 00:00:00'], 'data': [22, 14, 1, 4, 6]}, 'label': 'DateOfService', 'count': 5, 'min': '', 'max': ''}, {'values': {'label': ['7323A5D3-9249-4F98-9E04-2A53151C8290', '4BA375FB-3B76-46E9-AAF9-EEE2897F1E5E', 'A724E957-9B9A-435F-A593-D3272B479E7B', '7A9BF809-317C-435B-9CF4-868B78F71063', '3DD9EBAC-8AE9-47BF-A18C-5B30FAE84F85', '764637D6-5A4A-419F-8DFE-19D196A19B1D', 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', '0AC82717-3661-4548-8119-193431DEDEDA', '69294C2D-F7DC-473E-AA9E-53A44DC9B067', '03151B5D-685B-4211-B01F-7BBB1C4A8A49', '530C8C66-2D52-4067-8526-D4F3C94372F0'], 'data': [2, 8, 5, 4, 2, 4, 1, 7, 4, 6, 4]}, 'label': 'ProviderID', 'count': 11, 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3'}, {'values': {'label': ['92014', '92134', '92250', '99204', '92012', '92226', '92225'], 'data': [16, 18, 1, 3, 4, 1, 4]}, 'label': 'CPTCode', 'count': 7, 'min': '92012', 'max': '99204'}, {'values': {'label': ['H33.313', 'H35.3132', 'H43.821', 'E11.3513', 'H35.3122', 'H34.8110', 'H35.3221', 'H34.8310', 'H40.1231', 'H25.89', 'H35.373', 'G45.9', 'E11.3312', 'H35.3211', 'E11.3522', 'E11.9', 'H35.3212', 'H35.3112', 'H35.3210', 'H04.123', 'H33.002', 'H35.352', 'H35.413', 'E11.3413', 'H35.033', 'H35.371', 'H53.9', 'H35.3213', 'E11.3313', 'H33.302', 'H43.813', 'H35.3231', 'H40.1130', 'H33.321', 'H35.712'], 'data': [1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1]}, 'label': 'pdiagnosis1', 'count': 35, 'min': 'E11.3312', 'max': 'H53.9'}, {'values': {'label': ['D31.31', 'H33.313', 'H35.372', 'H35.3132', 'H35.3134', 'H35.3122', 'H35.3221', 'B39.4', 'H35.342', 'H35.373', 'H43.11', 'H35.3211', 'H34.8322', 'H44.23', 'E11.9', 'H35.461', 'H35.052', 'H33.002', 'H25.12', 'H35.352', 'H35.413', 'H35.063', 'E11.3291', 'H35.63', 'H35.033', 'H35.371', 'H35.70', 'H43.12', 'H35.463', 'H43.812', 'E11.3313', 'H43.813', 'Z96.1'], 'data': [2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 3, 1, 1, 1, 2, 1, 1, 1]}, 'label': 'pdiagnosis2', 'count': 33, 'min': 'B39.4', 'max': 'Z96.1'}, {'values': {'label': ['Pulmonary ', 'Orthopedic', 'Cardiology', 'Emergency'], 'data': [20, 3, 5, 19]}, 'label': 'DepartmentCategory', 'count': 4, 'min': 'Cardiology', 'max': 'Pulmonary '}, {'values': {'label': ['Ophtamology'], 'data': [47]}, 'label': 'SpecialityCategory', 'count': 1, 'min': 'Ophtamology', 'max': 'Ophtamology'}, {'values': {'label': ['0.0 - 13.8', '13.8 - 27.6', '27.6 - 41.4', '41.4 - 55.2', '55.2 - 69.0', '69.0 - 82.8', '82.8 - 96.6', '96.6 - 110.4', '110.4 - 124.2', '124.2 - 138.0'], 'data': [12, 12, 12, 6, 1, 2, 0, 1, 0, 1]}, 'label': 'days_to_first_post', 'count': 10, 'min': 0, 'max': 138}, {'values': {'label': ['75F80B5C-AD0E-4A1C-946C-07AEB69B7F9B', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', '2AF763CD-E2CF-4E69-80E5-B00EC12B032F', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'AD96772F-54C7-4750-86B1-1BD565B55244', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '91A3F98C-94F0-4427-8B42-106B6D6C707E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '315C773B-4EF1-4164-AB43-6817D993329F', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'B9F6F179-DD6F-486B-98EE-C207EE60842E', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4'], 'data': [1, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 5, 1, 2, 1, 16, 2, 2, 1, 2]}, 'label': 'PayerId', 'count': 20, 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52'}, {'values': {'label': ['0.0 - 4.6', '4.6 - 9.2', '9.2 - 13.8', '13.8 - 18.4', '18.4 - 23.0', '23.0 - 27.6', '27.6 - 32.2', '32.2 - 36.8', '36.8 - 41.4', '41.4 - 46.0'], 'data': [5, 5, 4, 5, 4, 5, 5, 4, 5, 5]}, 'label': 'ChargeDetailID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 46}, {'values': {'label': ['1', '0'], 'data': [21, 26]}, 'label': 'Gender_stringindexer_transform', 'count': 2, 'min': '0', 'max': '1'}, {'values': {'label': ['3', '5', '4', '2'], 'data': [9, 17, 17, 4]}, 'label': 'DateOfEntry_dayofmonth', 'count': 4, 'min': '2', 'max': '5'}, {'values': {'label': ['1'], 'data': [47]}, 'label': 'DateOfEntry_month', 'count': 1, 'min': '1', 'max': '1'}, {'values': {'label': ['2020'], 'data': [47]}, 'label': 'DateOfEntry_year', 'count': 1, 'min': '2020', 'max': '2020'}, {'values': {'label': ['3', '4', '8', '29', '2'], 'data': [14, 22, 1, 4, 6]}, 'label': 'DateOfService_dayofmonth', 'count': 5, 'min': '2', 'max': '29'}, {'values': {'label': ['12', '1'], 'data': [5, 42]}, 'label': 'DateOfService_month', 'count': 2, 'min': '1', 'max': '12'}, {'values': {'label': ['2019', '2020'], 'data': [5, 42]}, 'label': 'DateOfService_year', 'count': 2, 'min': '2019', 'max': '2020'}, {'values': {'label': ['0.0 - 1.0', '1.0 - 2.0', '2.0 - 3.0', '3.0 - 4.0', '4.0 - 5.0', '5.0 - 6.0', '6.0 - 7.0', '7.0 - 8.0', '8.0 - 9.0', '9.0 - 10.0'], 'data': [8, 7, 6, 5, 4, 4, 4, 4, 2, 3]}, 'label': 'ProviderID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 10}, {'values': {'label': ['0.0', '1.0', '4.0', '3.0', '2.0', '6.0', '5.0'], 'data': [18, 16, 3, 4, 4, 1, 1]}, 'label': 'CPTCode_stringindexer_transform', 'count': 7, 'min': '0.0', 'max': '6.0'}, {'values': {'label': ['0.0 - 3.4', '3.4 - 6.8', '6.8 - 10.2', '10.2 - 13.6', '13.6 - 17.0', '17.0 - 20.4', '20.4 - 23.8', '23.8 - 27.2', '27.2 - 30.6', '30.6 - 34.0'], 'data': [14, 5, 4, 3, 3, 4, 3, 4, 3, 4]}, 'label': 'pdiagnosis1_stringindexer_transform', 'count': 10, 'min': 0, 'max': 34}, {'values': {'label': ['0.0 - 3.2', '3.2 - 6.4', '6.4 - 9.6', '9.6 - 12.8', '12.8 - 16.0', '16.0 - 19.2', '19.2 - 22.4', '22.4 - 25.6', '25.6 - 28.8', '28.8 - 32.0'], 'data': [13, 6, 5, 3, 3, 4, 3, 3, 3, 4]}, 'label': 'pdiagnosis2_stringindexer_transform', 'count': 10, 'min': 0, 'max': 32}, {'values': {'label': ['1', '3', '2', '0'], 'data': [19, 3, 5, 20]}, 'label': 'DepartmentCategory_stringindexer_transform', 'count': 4, 'min': '0', 'max': '3'}, {'values': {'label': ['0'], 'data': [47]}, 'label': 'SpecialityCategory_stringindexer_transform', 'count': 1, 'min': '0', 'max': '0'}, {'values': {'label': ['0.0 - 1.9', '1.9 - 3.8', '3.8 - 5.7', '5.7 - 7.6', '7.6 - 9.5', '9.5 - 11.4', '11.4 - 13.3', '13.3 - 15.2', '15.2 - 17.1', '17.1 - 19.0'], 'data': [21, 6, 4, 4, 2, 2, 2, 2, 2, 2]}, 'label': 'PayerId_stringindexer_transform', 'count': 10, 'min': 0, 'max': 19}], 'path_ManualSample': '/FileStore/platform/sampleData/60824e962e36424d53db3377_ManualFE/part-00000-tid-8790912989024766106-9063b124-4e5f-4e0c-aa79-e07bb5fdcc72-482370-1-c000.csv'}, 'selectedFE': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '0085689C-C880-444A-AD7E-ACAF2BD1C2B7', 'max': 'FD457EC1-8819-47D0-B28D-3BCF4DB749C2', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '71.83', 'stddev': '15.61', 'min': '34', 'max': '98', 'missing': '0'}, 'updatedLabel': 'Age'}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateOfEntry'}, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateOfService'}, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '92546.21', 'stddev': '1758.91', 'min': '92012', 'max': '99204', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'E11.3312', 'max': 'H53.9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'B39.4', 'max': 'Z96.1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'days_to_first_post', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '47', 'mean': '30.79', 'stddev': '26.24', 'min': '0', 'max': '138', 'missing': '0'}, 'updatedLabel': 'days_to_first_post'}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '47', 'mean': '', 'stddev': '', 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '23.0', 'stddev': '13.71', 'min': '0.0', 'max': '46.0', 'missing': '0'}, 'updatedLabel': 'ChargeDetailID_stringinde...'}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}, 'updatedLabel': 'Gender_stringindexer_tran...'}, {'feature': 'DateOfEntry_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '4.0', 'stddev': '0.96', 'min': '2', 'max': '5', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_dayofmonth'}, {'feature': 'DateOfEntry_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '1.0', 'stddev': '0.0', 'min': '1', 'max': '1', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_month'}, {'feature': 'DateOfEntry_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateOfEntry_year'}, {'feature': 'DateOfService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '5.66', 'stddev': '7.26', 'min': '2', 'max': '29', 'missing': '0'}, 'updatedLabel': 'DateOfService_dayofmonth'}, {'feature': 'DateOfService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2.17', 'stddev': '3.43', 'min': '1', 'max': '12', 'missing': '0'}, 'updatedLabel': 'DateOfService_month'}, {'feature': 'DateOfService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '47', 'mean': '2019.89', 'stddev': '0.31', 'min': '2019', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateOfService_year'}, {'feature': 'ProviderID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '3.53', 'stddev': '2.89', 'min': '0.0', 'max': '10.0', 'missing': '0'}, 'updatedLabel': 'ProviderID_stringindexer_...'}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '1.26', 'stddev': '1.5', 'min': '0.0', 'max': '6.0', 'missing': '0'}, 'updatedLabel': 'CPTCode_stringindexer_tra...'}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '13.06', 'stddev': '11.16', 'min': '0.0', 'max': '34.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis1_stringindexer...'}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '12.13', 'stddev': '10.16', 'min': '0.0', 'max': '32.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis2_stringindexer...'}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.81', 'stddev': '0.88', 'min': '0', 'max': '3', 'missing': '0'}, 'updatedLabel': 'DepartmentCategory_string...'}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '47', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}, 'updatedLabel': 'SpecialityCategory_string...'}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '47', 'mean': '4.79', 'stddev': '5.79', 'min': '0.0', 'max': '19.0', 'missing': '0'}, 'updatedLabel': 'PayerId_stringindexer_tra...'}], 'autoFE_flag': True, 'source_attributes': {'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\r\n\t\t\t\tSub2.PayerId\r\n\r\nFROM \r\n\r\n(\r\n\r\n\t\tSELECT \r\n\r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\r\n\t\tpp.ProviderID,\r\n\r\n\t\tcptc.CPTCode,\r\n\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\r\n\t\td.DepartmentCategory,\r\n\r\n\t\ts.SpecialityCategory,\r\n\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\r\n\t\tfrom Transactions t\r\n\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\r\n\t\tand t.DateOfEntry between '1/1/2020' and '1/05/2020'  \r\n\r\n) as sub2\r\n\r\nINNER JOIN\r\n\r\n(\r\n\r\n\t\tSELECT\r\n\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\r\n\t\t\t t.DateOfEntry,\r\n\r\n\t\t\t count(*) as total_transactions\r\n\r\n\t\tfrom Transactions t\r\n\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\r\n\t\tWHERE t.Amount > 0.1 \r\n\r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n\r\n) as sub1\r\n\r\n on sub1.cid = sub2.ChargeDetailID\r\n\r\n where sub2.Gender is not NULL\r\n\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\r\n\t\t\t\tSub2.PayerId", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}, 'modeltrainingpercent': '100', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}, 'dbfsFileName': 'dpsinktest', 'directory_path': '/tmp'}")
	#sinkPostExecutionHook(dbfssink)

except Exception as ex: 
	logging.error(ex)
